# **Running the BAO fit for a given dataset and settings**

This notebook shows how to run a BAO fit

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. Single BAO fit

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from utils_data import WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

# 1. Arguments
class Args:
    def __init__(self):
        self.include_wiggles = "y"
        self.dataset = "DESY6"
        # self.dataset = "DESY6_dec<-23.5"
        # self.dataset = "DESY6_dec>-23.5"
        self.weight_type = 1
        self.mock_id = "mean" # it will not be used...
        self.nz_flag = "fid"
        # self.nz_flag = "clusteringz"
        self.cov_type = "cosmolike"
        self.cosmology_template = "planck"
        self.cosmology_covariance = "planck"
        self.delta_theta = 0.2
        self.theta_min = {bin_z: 0.5 for bin_z in range(6)}
        self.theta_max = {bin_z: 5 for bin_z in range(6)}
        self.n_broadband = 3
        self.bins_removed = []
        self.diag_only = "n"
        self.remove_crosscov = "n"
        self.alpha_min = 0.8
        self.alpha_max = 1.2
        self.base_path = None
args = Args()
args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"

fit_galaxybias = True

if fit_galaxybias:
    # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id="mean",
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=0.2,
        theta_min = {bin_z: 0.5 for bin_z in range(6)},
        theta_max = {bin_z: 2 for bin_z in range(6)},
        bins_removed=[],
        diag_only="n",
        remove_crosscov="n",
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    nbins = len(wtheta_data)
    wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])

    # 3. Get the model wtheta (for the linear galaxy bias fit)
    wtheta_model_galaxybias = WThetaModelGalaxyBias(
        include_wiggles="",
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        base_path=args.base_path,
    )

    # 4. Fit the linear galaxy bias
    popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
    galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}

    print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")

else:
    galaxy_bias = {bin_z: 2 for bin_z in range(6)}

# 2. Load data w(theta) and covariance
wtheta_data_covariance = WThetaDataCovariance(
    dataset=args.dataset,
    weight_type=args.weight_type,
    mock_id=args.mock_id,
    nz_flag=args.nz_flag, # this is only needed to get nbins...
    cov_type=args.cov_type,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    bins_removed=args.bins_removed,
    diag_only=args.diag_only,
    remove_crosscov=args.remove_crosscov,
)
theta_data, wtheta_data, cov = wtheta_data_covariance.process()

# 3. Get the model wtheta
wtheta_model = WThetaModel(
    include_wiggles=args.include_wiggles,
    dataset=args.dataset,
    nz_flag=args.nz_flag,
    cosmology_template=args.cosmology_template,
    n_broadband=args.n_broadband,
    galaxy_bias=galaxy_bias,
    base_path=args.base_path,
)

# 4. BAO fit initializer. This basically creates the path to save the results
baofit_initializer = BAOFitInitializer(
    include_wiggles=args.include_wiggles,
    dataset=args.dataset,
    weight_type=args.weight_type,
    mock_id=args.mock_id,
    nz_flag=args.nz_flag,
    cov_type=args.cov_type,
    cosmology_template=args.cosmology_template,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    n_broadband=args.n_broadband,
    bins_removed=args.bins_removed,
    alpha_min=args.alpha_min,
    alpha_max=args.alpha_max,
    verbose=True,
    base_path=args.base_path,
)

# 5. Run the BAO fit
baofit = BAOFit(
    baofit_initializer=baofit_initializer,
    wtheta_model=wtheta_model,
    theta_data=theta_data,
    wtheta_data=wtheta_data,
    cov=cov,
    close_fig=False,
    use_multiprocessing=True,
    n_cpu=256
)
alpha_best, err_alpha, chi2_best, dof = baofit.fit()


## 2. Multiple BAO fits (varying settings)

In [ ]:
import numpy as np
from itertools import product
from scipy.optimize import curve_fit
from utils_data import WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

dataset_list = ["DESY6"]
# dataset_list = ["DESY6_dec<-23.5", "DESY6_dec>-23.5"]
# dataset_list = ["DESY6_DR1tiles_noDESI"]
# dataset_list = ["DESY6_DR1tiles_DESIonly"]
include_wiggles_list = ["y", "n"]
weight_type_list = [1, 0]
bins_removed_list = [[], [0], [1], [2], [3], [4], [5], [1,2,3,4,5], [0,2,3,4,5], 
                     [0,1,3,4,5], [0,1,2,4,5], [0,1,2,3,5], [0,1,2,3,4]]

for dataset, include_wiggles, weight_type, bins_removed in product(dataset_list, include_wiggles_list, weight_type_list, bins_removed_list):
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = include_wiggles
            self.dataset = dataset
            self.weight_type = weight_type
            self.mock_id = "mean" # it will not be used...
            self.nz_flag = "fid"
            self.cov_type = "cosmolike"
            self.cosmology_template = "planck"
            self.cosmology_covariance = "planck"
            self.delta_theta = 0.2
            self.theta_min = {bin_z: 0.5 for bin_z in range(6)}
            self.theta_max = {bin_z: 5 for bin_z in range(6)}
            self.n_broadband = 3
            self.bins_removed = bins_removed
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
    
    fit_galaxybias = True
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            dataset=args.dataset,
            weight_type=args.weight_type,
            mock_id="mean",
            nz_flag=args.nz_flag, # this is only needed to get nbins...
            cov_type=args.cov_type,
            cosmology_covariance=args.cosmology_covariance,
            delta_theta=0.2,
            theta_min={bin_z: 0.5 for bin_z in range(6)},
            theta_max={bin_z: 2 for bin_z in range(6)},
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        nbins = len(wtheta_data)
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            include_wiggles="",
            dataset=args.dataset,
            nz_flag=args.nz_flag,
            cosmology_template=args.cosmology_template,
            base_path=args.base_path,
        )
    
        # 5. Fit the linear galaxy bias
        popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    else:
        galaxy_bias = {bin_z: 2 for bin_z in range(6)}
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        bins_removed=args.bins_removed,
        diag_only=args.diag_only,
        remove_crosscov=args.remove_crosscov,
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModel(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        n_broadband=args.n_broadband,
        galaxy_bias=galaxy_bias,
        base_path=args.base_path,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=True,
        base_path=args.base_path,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=True,
        use_multiprocessing=True,
        n_cpu=256
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()
    

## 3. Multiple BAO fits (DES COLA mocks)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

dataset = "DESY6_COLA"
nz_flag = "mocks"

mock_id_list = range(1000)
mock_id_list = ["mean"]

nz = RedshiftDistributions(dataset=dataset, nz_flag=nz_flag)

dynamical_theta_ranges = False

if dynamical_theta_ranges:
    from cosmoprimo.fiducial import DESI
    
    cosmo_DESI = DESI()
    rd_mpc = cosmo_DESI.rs_drag / cosmo_DESI.h
    
    def angular_bao_scale_deg(z):   
        DA = cosmo_DESI.comoving_angular_distance(z) # Mpc
        theta_rad = rd_mpc / ((1 + z) * DA)
        return np.degrees(theta_rad)
    
    theta_width = 2.0  # fixed width
    
    theta_min = {}
    theta_max = {}
    
    for bin_z in range(nz.nbins):
        zeff = np.mean(nz.z_edges[bin_z])
        theta_bao = angular_bao_scale_deg(zeff) + 1 * (zeff - 0.4)
        theta_min[bin_z] = theta_bao - theta_width / 2
        theta_max[bin_z] = theta_bao + theta_width / 2

else:
    theta_min = {bin_z: 0.5 for bin_z in range(nz.nbins)}
    theta_max = {bin_z: 5 for bin_z in range(nz.nbins)}

for mock_id in mock_id_list:
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = "y"
            self.dataset = dataset
            self.weight_type = 1 # it will not be used...
            self.mock_id = mock_id
            self.nz_flag = nz_flag
            self.cov_type = "cosmolike"
            self.cosmology_template = "mice"
            self.cosmology_covariance = "mice"
            self.delta_theta = 0.2
            self.theta_min = theta_min
            self.theta_max = theta_max
            self.n_broadband = 3
            self.bins_removed = []
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"

    if mock_id == mock_id_list[0]:
        fit_galaxybias = True
    else:
        fit_galaxybias = False

    if mock_id == "mean":
        close_fig = False
    else:
        close_fig = True
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            dataset=args.dataset,
            weight_type=args.weight_type,
            mock_id="mean",
            nz_flag=args.nz_flag, # this is only needed to get nbins...
            cov_type=args.cov_type,
            cosmology_covariance=args.cosmology_covariance,
            delta_theta=0.2,
            theta_min={bin_z: 0.5 for bin_z in range(nz.nbins)},
            theta_max={bin_z: 2 for bin_z in range(nz.nbins)},
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        nbins = len(wtheta_data)
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            include_wiggles="",
            dataset=args.dataset,
            nz_flag=args.nz_flag,
            cosmology_template=args.cosmology_template,
            base_path=args.base_path,
        )
    
        # 4. Fit the linear galaxy bias
        popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        bins_removed=args.bins_removed,
        diag_only=args.diag_only,
        remove_crosscov=args.remove_crosscov,
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModel(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        n_broadband=args.n_broadband,
        galaxy_bias=galaxy_bias,
        base_path=args.base_path,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=True,
        base_path=args.base_path,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=close_fig,
        use_multiprocessing=True,
        n_cpu=256
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()


## 4. Multiple BAO fits (DESI EZ mocks)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

dataset = "DESIY1_LRG_EZ"
nz_flag = "mocks"

# mock_id_list = range(1000)
mock_id_list = ["mean"]

nz = RedshiftDistributions(dataset=dataset, nz_flag=nz_flag)

dynamical_theta_ranges = True

if dynamical_theta_ranges:
    from cosmoprimo.fiducial import DESI
    
    cosmo_DESI = DESI()
    rd_mpc = cosmo_DESI.rs_drag / cosmo_DESI.h
    
    def angular_bao_scale_deg(z):   
        DA = cosmo_DESI.comoving_angular_distance(z) # Mpc
        theta_rad = rd_mpc / ((1 + z) * DA)
        return np.degrees(theta_rad)
    
    theta_width = 3.0  # fixed width
    
    theta_min = {}
    theta_max = {}
    
    for bin_z in range(nz.nbins):
        zeff = np.mean(nz.z_edges[bin_z])
        theta_bao = angular_bao_scale_deg(zeff) + 1 * (zeff - 0.4)
        theta_min[bin_z] = theta_bao - theta_width / 2
        theta_max[bin_z] = theta_bao + theta_width / 2

else:
    theta_min = {bin_z: 0.5 for bin_z in range(nz.nbins)}
    theta_max = {bin_z: 8 for bin_z in range(nz.nbins)}

for mock_id in mock_id_list:
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = "y"
            self.dataset = dataset
            self.weight_type = 1 # it will not be used...
            self.mock_id = mock_id
            self.nz_flag = nz_flag
            self.cov_type = "mocks"
            self.cosmology_template = "desifid"
            self.cosmology_covariance = "desifid"
            self.delta_theta = 0.2
            self.theta_min = theta_min
            self.theta_max = theta_max
            self.n_broadband = 3
            self.bins_removed = []
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"

    if mock_id == mock_id_list[0]:
        fit_galaxybias = True
    else:
        fit_galaxybias = False

    if mock_id == "mean":
        close_fig = False
    else:
        close_fig = True
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            dataset=args.dataset,
            weight_type=args.weight_type,
            mock_id="mean",
            nz_flag=args.nz_flag, # this is only needed to get nbins...
            cov_type=args.cov_type,
            cosmology_covariance=args.cosmology_covariance,
            delta_theta=0.2,
            theta_min={bin_z: 0.5 for bin_z in range(nz.nbins)},
            theta_max={bin_z: 2 for bin_z in range(nz.nbins)},
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        nbins = len(wtheta_data)
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            include_wiggles="",
            dataset=args.dataset,
            nz_flag=args.nz_flag,
            cosmology_template=args.cosmology_template,
            base_path=args.base_path,
        )
    
        # 4. Fit the linear galaxy bias
        popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        bins_removed=args.bins_removed,
        diag_only=args.diag_only,
        remove_crosscov=args.remove_crosscov,
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModel(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        n_broadband=args.n_broadband,
        galaxy_bias=galaxy_bias,
        base_path=args.base_path,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=True,
        base_path=args.base_path,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=close_fig,
        use_multiprocessing=True,
        n_cpu=256
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()


## 5. Multiple BAO fits (DESI Abacus mocks)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModel, BAOFitInitializer, BAOFit

dataset = "DESIY1_LRG_Abacus_complete"
nz_flag = "mocks"

# mock_id_list = range(1000)
mock_id_list = ["mean"]

nz = RedshiftDistributions(dataset=dataset, nz_flag=nz_flag)

dynamical_theta_ranges = True

if dynamical_theta_ranges:
    from cosmoprimo.fiducial import DESI
    
    cosmo_DESI = DESI()
    rd_mpc = cosmo_DESI.rs_drag / cosmo_DESI.h
    
    def angular_bao_scale_deg(z):   
        DA = cosmo_DESI.comoving_angular_distance(z) # Mpc
        theta_rad = rd_mpc / ((1 + z) * DA)
        return np.degrees(theta_rad)
    
    theta_width = 3.0  # fixed width
    
    theta_min = {}
    theta_max = {}
    
    for bin_z in range(nz.nbins):
        zeff = np.mean(nz.z_edges[bin_z])
        theta_bao = angular_bao_scale_deg(zeff) + 1 * (zeff - 0.4)
        theta_min[bin_z] = theta_bao - theta_width / 2
        theta_max[bin_z] = theta_bao + theta_width / 2

else:
    theta_min = {bin_z: 0.5 for bin_z in range(nz.nbins)}
    theta_max = {bin_z: 8 for bin_z in range(nz.nbins)}

for mock_id in mock_id_list:
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = "y"
            self.dataset = dataset
            self.weight_type = 1 # it will not be used...
            self.mock_id = mock_id
            self.nz_flag = nz_flag
            self.cov_type = "mocks"
            self.cosmology_template = "desifid"
            self.cosmology_covariance = "desifid"
            self.delta_theta = 0.2
            self.theta_min = theta_min
            self.theta_max = theta_max
            self.n_broadband = 3
            self.bins_removed = []
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"

    if mock_id == mock_id_list[0]:
        fit_galaxybias = True
    else:
        fit_galaxybias = False

    if mock_id == "mean":
        close_fig = False
    else:
        close_fig = True
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            dataset=args.dataset,
            weight_type=args.weight_type,
            mock_id="mean",
            nz_flag=args.nz_flag, # this is only needed to get nbins...
            cov_type=args.cov_type,
            cosmology_covariance=args.cosmology_covariance,
            delta_theta=0.2,
            theta_min={bin_z: 0.5 for bin_z in range(nz.nbins)},
            theta_max={bin_z: 2 for bin_z in range(nz.nbins)},
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        nbins = len(wtheta_data)
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            include_wiggles="",
            dataset=args.dataset,
            nz_flag=args.nz_flag,
            cosmology_template=args.cosmology_template,
            base_path=args.base_path,
        )
    
        # 4. Fit the linear galaxy bias
        popt, pcov = curve_fit(wtheta_model_galaxybias.get_wtheta_function(), theta_data, wtheta_data_concatenated, np.ones(nbins), bounds=([0] * nbins, [3] * nbins), sigma=cov, absolute_sigma=False)
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag, # this is only needed to get nbins...
        cov_type=args.cov_type,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        bins_removed=args.bins_removed,
        diag_only=args.diag_only,
        remove_crosscov=args.remove_crosscov,
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModel(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        n_broadband=args.n_broadband,
        galaxy_bias=galaxy_bias,
        base_path=args.base_path,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=True,
        base_path=args.base_path,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=close_fig,
        use_multiprocessing=True,
        n_cpu=256
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()


In [ ]:
@ARTICLE{deputter,
       author = {{de Putter}, Roland and {Dor{\'e}}, Olivier},
        title = "{Designing an inflation galaxy survey: How to measure {\ensuremath{\sigma}} (f$_{NL}$){\ensuremath{\sim}}1 using scale-dependent galaxy bias}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2017,
        month = jun,
       volume = {95},
       number = {12},
          eid = {123513},
        pages = {123513},
          doi = {10.1103/PhysRevD.95.123513},
archivePrefix = {arXiv},
       eprint = {1412.3854},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2017PhRvD..95l3513D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@ARTICLE{DESI,
       author = {{DESI Collaboration}},
        title = "{The DESI Experiment Part I: Science,Targeting, and Survey Design}",
      journal = {arXiv e-prints},
     keywords = {Astrophysics - Instrumentation and Methods for Astrophysics, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2016,
        month = oct,
          eid = {arXiv:1611.00036},
        pages = {arXiv:1611.00036},
          doi = {10.48550/arXiv.1611.00036},
archivePrefix = {arXiv},
       eprint = {1611.00036},
 primaryClass = {astro-ph.IM},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2016arXiv161100036D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@ARTICLE{DESDR1,
       author = {{Dark Energy Survey Collaboration}},
        title = "{The Dark Energy Survey: Data Release 1}",
      journal = {\apjs},
     keywords = {astronomical databases: miscellaneous, catalogs, cosmology: observations, surveys, techniques: image processing, techniques: photometric, Astrophysics - Instrumentation and Methods for Astrophysics, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies, Astrophysics - Solar and Stellar Astrophysics},
         year = 2018,
        month = dec,
       volume = {239},
       number = {2},
          eid = {18},
        pages = {18},
          doi = {10.3847/1538-4365/aae9f0},
archivePrefix = {arXiv},
       eprint = {1801.03181},
 primaryClass = {astro-ph.IM},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2018ApJS..239...18A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

%author = {{Abbott}, T.~M.~C. and {Adam{\'o}w}, M. and {Aguena}, M. and {Allam}, S. and {Amon}, A. and {Annis}, J. and {Avila}, S. and {Bacon}, D. and {Banerji}, M. and {Bechtol}, K. and {Becker}, M.~R. and {Bernstein}, G.~M. and {Bertin}, E. and {Bhargava}, S. and {Bridle}, S.~L. and {Brooks}, D. and {Burke}, D.~L. and {Carnero Rosell}, A. and {Carrasco Kind}, M. and {Carretero}, J. and {Castander}, F.~J. and {Cawthon}, R. and {Chang}, C. and {Choi}, A. and {Conselice}, C. and {Costanzi}, M. and {Crocce}, M. and {da Costa}, L.~N. and {Davis}, T.~M. and {De Vicente}, J. and {DeRose}, J. and {Desai}, S. and {Diehl}, H.~T. and {Dietrich}, J.~P. and {Drlica-Wagner}, A. and {Eckert}, K. and {Elvin-Poole}, J. and {Everett}, S. and {Evrard}, A.~E. and {Ferrero}, I. and {Fert{\'e}}, A. and {Flaugher}, B. and {Fosalba}, P. and {Friedel}, D. and {Frieman}, J. and {Garc{\'\i}a-Bellido}, J. and {Gaztanaga}, E. and {Gelman}, L. and {Gerdes}, D.~W. and {Giannantonio}, T. and {Gill}, M.~S.~S. and {Gruen}, D. and {Gruendl}, R.~A. and {Gschwend}, J. and {Gutierrez}, G. and {Hartley}, W.~G. and {Hinton}, S.~R. and {Hollowood}, D.~L. and {Honscheid}, K. and {Huterer}, D. and {James}, D.~J. and {Jeltema}, T. and {Johnson}, M.~D. and {Kent}, S. and {Kron}, R. and {Kuehn}, K. and {Kuropatkin}, N. and {Lahav}, O. and {Li}, T.~S. and {Lidman}, C. and {Lin}, H. and {MacCrann}, N. and {Maia}, M.~A.~G. and {Manning}, T.~A. and {Maloney}, J.~D. and {March}, M. and {Marshall}, J.~L. and {Martini}, P. and {Melchior}, P. and {Menanteau}, F. and {Miquel}, R. and {Morgan}, R. and {Myles}, J. and {Neilsen}, E. and {Ogando}, R.~L.~C. and {Palmese}, A. and {Paz-Chinch{\'o}n}, F. and {Petravick}, D. and {Pieres}, A. and {Plazas}, A.~A. and {Pond}, C. and {Rodriguez-Monroy}, M. and {Romer}, A.~K. and {Roodman}, A. and {Rykoff}, E.~S. and {Sako}, M. and {Sanchez}, E. and {Santiago}, B. and {Scarpine}, V. and {Serrano}, S. and {Sevilla-Noarbe}, I. and {Smith}, J. Allyn and {Smith}, M. and {Soares-Santos}, M. and {Suchyta}, E. and {Swanson}, M.~E.~C. and {Tarle}, G. and {Thomas}, D. and {To}, C. and {Tremblay}, P.~E. and {Troxel}, M.~A. and {Tucker}, D.~L. and {Turner}, D.~J. and {Varga}, T.~N. and {Walker}, A.~R. and {Wechsler}, R.~H. and {Weller}, J. and {Wester}, W. and {Wilkinson}, R.~D. and {Yanny}, B. and {Zhang}, Y. and {Nikutta}, R. and {Fitzpatrick}, M. and {Jacques}, A. and {Scott}, A. and {Olsen}, K. and {Huang}, L. and {Herrera}, D. and {Juneau}, S. and {Nidever}, D. and {Weaver}, B.~A. and {Adean}, C. and {Correia}, V. and {de Freitas}, M. and {Freitas}, F.~N. and {Singulani}, C. and {Vila-Verde}, G. and {Linea Science Server}},

@ARTICLE{DESDR2,
       author = {{Dark Energy Survey Collaboration}},
        title = "{The Dark Energy Survey Data Release 2}",
      journal = {\apjs},
     keywords = {Dark energy, Cosmology, Extragalactic astronomy, Surveys, Redshift surveys, Optical astronomy, Near infrared astronomy, Astrophysics - Instrumentation and Methods for Astrophysics, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies, Astrophysics - Solar and Stellar Astrophysics},
         year = 2021,
        month = aug,
       volume = {255},
       number = {2},
          eid = {20},
        pages = {20},
          doi = {10.3847/1538-4365/ac00b3},
archivePrefix = {arXiv},
       eprint = {2101.05765},
 primaryClass = {astro-ph.IM},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021ApJS..255...20A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}





@ARTICLE{PCF_Y3_BAO,
       author = {{Chan}, K.~C. and {Avila}, S. and {Carnero Rosell}, A. and {Ferrero}, I. and {Elvin-Poole}, J. and {Sanchez}, E. and {Camacho}, H. and {Porredon}, A. and {Crocce}, M. and DES},
        title = "{Dark Energy Survey Year 3 results: Measurement of the baryon acoustic oscillations with three-dimensional clustering}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = dec,
       volume = {106},
       number = {12},
          eid = {123502},
        pages = {123502},
          doi = {10.1103/PhysRevD.106.123502},
archivePrefix = {arXiv},
       eprint = {2210.05057},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.106l3502C},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{SOMPZ,
       author = {{Giannini}, G. and the DES},
        title = "{Dark Energy Survey Year 3 Results: Redshift Calibration of the MagLim Lens Sample from the combination of SOMPZ and clustering and its impact on Cosmology}",
      journal = {arXiv e-prints},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = sep,
          eid = {arXiv:2209.05853},
        pages = {arXiv:2209.05853},
          doi = {10.48550/arXiv.2209.05853},
archivePrefix = {arXiv},
       eprint = {2209.05853},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022arXiv220905853G},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




@ARTICLE{PCF_method_Y3,
       author = {{Chan}, Kwan Chuen and {Ferrero}, Ismael and {Avila}, Santiago and {Ross}, Ashley J. and {Crocce}, Martin and {Gazta{\~n}aga}, Enrique},
        title = "{Clustering with general photo-z uncertainties: application to Baryon Acoustic Oscillations}",
      journal = {\mnras},
     keywords = {cosmology: observations, (cosmology:) large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = apr,
       volume = {511},
       number = {3},
        pages = {3965-3982},
          doi = {10.1093/mnras/stac340},
archivePrefix = {arXiv},
       eprint = {2110.13332},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022MNRAS.511.3965C},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




@article{ross20,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.498.2354R},
	archiveprefix = {arXiv},
	author = {{Ross}, Ashley J. and others},
	date-added = {2023-07-25 20:03:14 -0700},
	date-modified = {2023-07-25 20:03:32 -0700},
	doi = {10.1093/mnras/staa2416},
	eprint = {2007.09000},
	journal = {\mnras},
	keywords = {catalogues, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	number = {2},
	pages = {2354-2371},
	primaryclass = {astro-ph.CO},
	title = {{The Completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: Large-scale structure catalogues for cosmological analysis}},
	volume = {498},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa2416}}

@article{raichoor21,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.500.3254R},
	archiveprefix = {arXiv},
	author = {{Raichoor}, Anand and others},
	date-added = {2023-07-25 20:02:31 -0700},
	date-modified = {2023-07-25 20:03:00 -0700},
	doi = {10.1093/mnras/staa3336},
	eprint = {2007.09007},
	journal = {\mnras},
	keywords = {galaxies: distances and redshifts, dark energy, distance scale, large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jan,
	number = {3},
	pages = {3254-3274},
	primaryclass = {astro-ph.CO},
	title = {{The completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: large-scale structure catalogues and measurement of the isotropic BAO between redshift 0.6 and 1.1 for the Emission Line Galaxy Sample}},
	volume = {500},
	year = 2021,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa3336}}

@article{reid16,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.1553R},
	archiveprefix = {arXiv},
	author = {{Reid}, Beth and others},
	date-added = {2023-07-25 20:01:53 -0700},
	date-modified = {2023-07-25 20:02:11 -0700},
	doi = {10.1093/mnras/stv2382},
	eprint = {1509.06529},
	journal = {\mnras},
	keywords = {cosmology: observations, (cosmology:) large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies},
	month = jan,
	number = {2},
	pages = {1553-1573},
	primaryclass = {astro-ph.CO},
	title = {{SDSS-III Baryon Oscillation Spectroscopic Survey Data Release 12: galaxy target selection and large-scale structure catalogues}},
	volume = {455},
	year = 2016,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stv2382}}

@article{porredon22,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.106j3530P},
	archiveprefix = {arXiv},
	author = {{Porredon}, A. and DES},
	date-added = {2023-07-25 20:01:09 -0700},
	date-modified = {2023-07-25 20:01:28 -0700},
	doi = {10.1103/PhysRevD.106.103530},
	eid = {103530},
	eprint = {2105.13546},
	journal = {\prd},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = nov,
	number = {10},
	pages = {103530},
	primaryclass = {astro-ph.CO},
	title = {{Dark Energy Survey Year 3 results: Cosmological constraints from galaxy clustering and galaxy-galaxy lensing using the MAGLIM lens sample}},
	volume = {106},
	year = 2022,
	bdsk-url-1 = {https://doi.org/10.1103/PhysRevD.106.103530}}

@ARTICLE{porredon21,
       author = {{Porredon}, A. and DES},
        title = "{Dark Energy Survey Year 3 results: Optimizing the lens sample in a combined galaxy clustering and galaxy-galaxy lensing analysis}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = feb,
       volume = {103},
       number = {4},
          eid = {043503},
        pages = {043503},
          doi = {10.1103/PhysRevD.103.043503},
archivePrefix = {arXiv},
       eprint = {2011.03411},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021PhRvD.103d3503P},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@article{cawthon22,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2022MNRAS.513.5517C},
	archiveprefix = {arXiv},
	author = {{Cawthon}, R. and others},
	date-added = {2023-07-25 19:59:08 -0700},
	date-modified = {2023-07-25 20:00:18 -0700},
	doi = {10.1093/mnras/stac1160},
	eprint = {2012.12826},
	journal = {\mnras},
	keywords = {surveys, galaxies: distances and redshifts, large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jul,
	number = {4},
	pages = {5517-5539},
	primaryclass = {astro-ph.CO},
	title = {{Dark Energy Survey Year 3 results: calibration of lens sample redshift distributions using clustering redshifts with BOSS/eBOSS}},
	volume = {513},
	year = 2022,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stac1160}}

@article{erler2015combination,
	author = {Erler, Jens},
	journal = {The European Physical Journal C},
	pages = {1--7},
	publisher = {Springer},
	title = {On the combination procedure of correlated errors},
	volume = {75},
	year = {2015}}

@article{y3-baomocks,
	author = {{Ferrero}, I. and DES},
        title = "{Dark Energy Survey Year 3 Results: Galaxy mock catalogs for BAO analysis}",
      journal = {\aap},
     keywords = {catalogs, large-scale structure of Universe, galaxies: distances and redshifts, Galaxy: halo, methods: numerical, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = dec,
       volume = {656},
          eid = {A106},
        pages = {A106},
          doi = {10.1051/0004-6361/202141744},
archivePrefix = {arXiv},
       eprint = {2107.04602},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021A&A...656A.106F},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{Mena23,
	author = {{Mena-Fern\'andez}, J. and DES},
	journal = {To be submitted to PRD},
	title = {{Dark Energy Survey Year 6 Results: Galaxy Sample for BAO Measurement}},
	year = {2024}
}

@ARTICLE{SNkey,
       author = {{Dark Energy Survey Collaboration}},
        title = "{The Dark Energy Survey: Cosmology Results With \raisebox{-0.5ex}\textasciitilde1500 New High-redshift Type Ia Supernovae Using The Full 5-year Dataset}",
      journal = {arXiv e-prints},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2024,
        month = jan,
        pages = {arXiv:2401.02929},
          doi = {10.48550/arXiv.2401.02929},
       eprint = {2401.02929},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2024arXiv240102929D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

%                  {Armstrong}, P. 
@ARTICLE{SNcosmo,
       author = {{Vincenzi}, M. and {Brout}, D. and 
                {DES}
                },
        title = "{The Dark Energy Survey Supernova Program: Cosmological Analysis and Systematic Uncertainties}",
      journal = {arXiv e-prints},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2024,
        month = jan,
          doi = {10.48550/arXiv.2401.02945},
       eprint = {2401.02945},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2024arXiv240102945V},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}





%@article{Y6_LSS_sample,
	author = {{Weaverdyck}, N. and {Sevilla-Noarbe}, I. and the DES},
	journal = {To be submitted to PRD},
	title = {{Dark Energy Survey Year 6 Results: galaxy clustering sample for combination with weak lensing}},
	year = {2024}
}

@article{Y6_LSS_sys,
	author = {{Weaverdyck}, N. and {Rodr{\'\i}guez-Monroy}, M. and the DES collaboration},
	journal = {To be submitted to PRD},
	title = {{Dark Energy Survey Year 6 Results: galaxy clustering measurements and observational systematics}},
	year = {2024}
}

@article{Y6_mask,
	author = {{Rodr{\'\i}guez-Monroy}, M. and {Weaverdyick}, N. and DES},
	journal = {To be submitted to PRD},
	title = {{Dark Energy Survey Year 6 Results: masking as a mitigation in galaxy clustering}},
	year = {2024}
}

@article{Y6_gold,
	author = {{Bechtol}, K. and others},
	journal = {To be submitted to ApJS},
	title = {{Dark Energy Survey Year 6 Results: Photometric Data Set for Cosmology}},
	year = {2023}
}


@article{y3-baosample,
	author = {{Carnero Rosell}, A. and DES},
        title = "{Dark Energy Survey Year 3 results: galaxy sample for BAO measurement}",
      journal = {\mnras},
     keywords = {catalogues, surveys, cosmology: observations, cosmology: large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = jan,
       volume = {509},
       number = {1},
        pages = {778-799},
          doi = {10.1093/mnras/stab2995},
archivePrefix = {arXiv},
       eprint = {2107.05477},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022MNRAS.509..778C},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{y3-baokp,
       author = {{Dark Energy Survey Collaboration} and others},
        title = "{Dark Energy Survey Year 3 results: A 2.7\% measurement of baryon acoustic oscillation distance scale at redshift 0.835}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = feb,
       volume = {105},
       number = {4},
          eid = {043512},
        pages = {043512},
          doi = {10.1103/PhysRevD.105.043512},
archivePrefix = {arXiv},
       eprint = {2107.04646},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.105d3512A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{y3-galaxyclustering,
	author = {{Rodr{\'\i}guez-Monroy}, M. and  DES },
        title = "{Dark Energy Survey Year 3 results: galaxy clustering and systematics treatment for lens galaxy samples}",
      journal = {\mnras},
     keywords = {cosmological parameters, cosmology: observations, dark energy, large-scale structure of the Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = apr,
       volume = {511},
       number = {2},
        pages = {2665-2687},
          doi = {10.1093/mnras/stac104},
archivePrefix = {arXiv},
       eprint = {2105.13540},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022MNRAS.511.2665R},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{y1_lss_sys_kp,
       author = {{Elvin-Poole}, J. and {DES}},
        title = "{Dark Energy Survey year 1 results: Galaxy clustering for combined probes}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2018,
        month = aug,
       volume = {98},
       number = {4},
          eid = {042006},
        pages = {042006},
          doi = {10.1103/PhysRevD.98.042006},
archivePrefix = {arXiv},
       eprint = {1708.01536},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2018PhRvD..98d2006E},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{sv_lss_sys,
       author = {{Crocce}, M. and {DES}},
        title = "{Galaxy clustering, photometric redshifts and diagnosis of systematics in the DES Science Verification data}",
      journal = {\mnras},
     keywords = {surveys, cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2016,
        month = feb,
       volume = {455},
       number = {4},
        pages = {4301-4324},
          doi = {10.1093/mnras/stv2590},
archivePrefix = {arXiv},
       eprint = {1507.05360},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.4301C},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




@article{y3-gold,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2021ApJS..254...24S},
	archiveprefix = {arXiv},
	author = {{Sevilla-Noarbe}, I. and DES},
	doi = {10.3847/1538-4365/abeb66},
	eid = {24},
	eprint = {2011.03407},
	journal = {\apjs},
	keywords = {Surveys, Catalogs, Observational cosmology, 1671, 205, 1146, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies, Astrophysics - Instrumentation and Methods for Astrophysics},
	month = jun,
	number = {2},
	pages = {24},
	primaryclass = {astro-ph.CO},
	title = {{Dark Energy Survey Year 3 Results: Photometric Data Set for Cosmology}},
	volume = {254},
	year = 2021,
	bdsk-url-1 = {https://doi.org/10.3847/1538-4365/abeb66}}

@article{y3-covariances,
	author = {{Friedrich}, O. and others},
        title = "{Dark Energy Survey year 3 results: covariance modelling and its impact on parameter estimation and quality of fit}",
      journal = {\mnras},
     keywords = {large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = dec,
       volume = {508},
       number = {3},
        pages = {3125-3165},
          doi = {10.1093/mnras/stab2384},
archivePrefix = {arXiv},
       eprint = {2012.08568},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.508.3125F},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{y3-3x2ptkp,
	author = {{Dark Energy Survey Collaboration}},
      title = "{Dark Energy Survey Year 3 results: Cosmological constraints from galaxy clustering and weak lensing}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = jan,
       volume = {105},
       number = {2},
          eid = {023520},
        pages = {023520},
          doi = {10.1103/PhysRevD.105.023520},
archivePrefix = {arXiv},
       eprint = {2105.13549},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.105b3520A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{2016MNRAS.460.1270D,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.460.1270D},
	archiveprefix = {arXiv},
	author = {{Dark Energy Survey Collaboration}},
	doi = {10.1093/mnras/stw641},
	eprint = {1601.00329},
	journal = {\mnras},
	keywords = {surveys, minor planets, asteroids: general, supernovae: general, Galaxy: general, galaxies: general, quasars: general, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies},
	month = aug,
	number = {2},
	pages = {1270-1299},
	primaryclass = {astro-ph.CO},
	title = {{The Dark Energy Survey: more than dark energy - an overview}},
	volume = {460},
	year = 2016,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw641}}

@article{1998ApJ...496..605E,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/1998ApJ...496..605E},
	archiveprefix = {arXiv},
	author = {{Eisenstein}, Daniel J. and {Hu}, Wayne},
	doi = {10.1086/305424},
	eprint = {astro-ph/9709112},
	journal = {\apj},
	keywords = {COSMOLOGY: THEORY, COSMOLOGY: DARK MATTER, COSMOLOGY: LARGE-SCALE STRUCTURE OF UNIVERSE, Cosmology: Theory, Cosmology: Dark Matter, Cosmology: Large-Scale Structure of Universe, Astrophysics},
	month = mar,
	number = {2},
	pages = {605-614},
	primaryclass = {astro-ph},
	title = {{Baryonic Features in the Matter Transfer Function}},
	volume = {496},
	year = 1998,
	bdsk-url-1 = {https://doi.org/10.1086/305424}}

@article{Hunter:2007,
	abstract = {Matplotlib is a 2D graphics package used for Python for
  application development, interactive scripting, and publication-quality
  image generation across user interfaces and operating systems.},
	author = {Hunter, J. D.},
	doi = {10.1109/MCSE.2007.55},
	journal = {Computing in Science \& Engineering},
	number = {3},
	pages = {90--95},
	publisher = {IEEE COMPUTER SOC},
	title = {Matplotlib: A 2D graphics environment},
	volume = {9},
	year = 2007,
	bdsk-url-1 = {https://doi.org/10.1109/MCSE.2007.55}}

@misc{OhioSupercomputerCenter1987,
	author = {OSC},
	howpublished = {\url{http://osc.edu/ark:/19495/f5s1ph73}},
	title = {Ohio Supercomputer Center},
	year = {1987}}

@article{2005ApJ...622..759G,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2005ApJ...622..759G},
	archiveprefix = {arXiv},
	author = {{G{\'o}rski}, K.~M. and {Hivon}, E. and {Banday}, A.~J. and {Wandelt}, B.~D. and {Hansen}, F.~K. and {Reinecke}, M. and {Bartelmann}, M.},
	doi = {10.1086/427976},
	eprint = {astro-ph/0409513},
	journal = {\apj},
	keywords = {Cosmology: Cosmic Microwave Background, Cosmology: Observations, Methods: Statistical, Astrophysics},
	month = apr,
	number = {2},
	pages = {759-771},
	primaryclass = {astro-ph},
	title = {{HEALPix: A Framework for High-Resolution Discretization and Fast Analysis of Data Distributed on the Sphere}},
	volume = {622},
	year = 2005,
	bdsk-url-1 = {https://doi.org/10.1086/427976}}

@article{2015AJ....150..150F,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015AJ....150..150F},
	archiveprefix = {arXiv},
	author = {{Flaugher}, B. and others},
	doi = {10.1088/0004-6256/150/5/150},
	eid = {150},
	eprint = {1504.02900},
	journal = {\aj},
	keywords = {atlases, catalogs, cosmology: observations, instrumentation: detectors, instrumentation: photometers, surveys, Astrophysics - Instrumentation and Methods for Astrophysics},
	month = nov,
	number = {5},
	pages = {150},
	primaryclass = {astro-ph.IM},
	title = {{The Dark Energy Camera}},
	volume = {150},
	year = 2015,
	bdsk-url-1 = {https://doi.org/10.1088/0004-6256/150/5/150}}

@ARTICLE{BAO_DECals,
       author = {{Sridhar}, Srivatsan and {Song}, Yong-Seon and {Ross}, Ashley J. and {Zhou}, Rongpu and {Newman}, Jeffrey A. and {Chuang}, Chia-HSun and {Blum}, Robert and {Gazta{\~n}aga}, Enrique and {Landriau}, Martin and {Prada}, Francisco},
        title = "{Clustering of LRGs in the DECaLS DR8 Footprint: Distance Constraints from Baryon Acoustic Oscillations Using Photometric Redshifts}",
      journal = {\apj},
     keywords = {Large-scale structure of the universe, Distance measure, Cosmology, High-redshift galaxies, Photometry, 902, 395, 343, 734, 1234, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2020,
        month = nov,
       volume = {904},
       number = {1},
          eid = {69},
        pages = {69},
          doi = {10.3847/1538-4357/abc0f0},
archivePrefix = {arXiv},
       eprint = {2005.13126},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2020ApJ...904...69S},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{DES05,
       author = {{The Dark Energy Survey Collaboration}},
        title = "{The Dark Energy Survey}",
      journal = {arXiv e-prints},
     keywords = {Astrophysics},
         year = 2005,
        month = oct,
          doi = {10.48550/arXiv.astro-ph/0510346},
       eprint = {astro-ph/0510346},
 primaryClass = {astro-ph},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2005astro.ph.10346T},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}





@article{RevModPhys.61.1,
	author = {Weinberg, Steven},
	doi = {10.1103/RevModPhys.61.1},
	issue = {1},
	journal = {Rev. Mod. Phys.},
	month = {Jan},
	numpages = {0},
	pages = {1--23},
	publisher = {American Physical Society},
	title = {The cosmological constant problem},
	url = {https://link.aps.org/doi/10.1103/RevModPhys.61.1},
	volume = {61},
	year = {1989},
	bdsk-url-1 = {https://link.aps.org/doi/10.1103/RevModPhys.61.1},
	bdsk-url-2 = {https://doi.org/10.1103/RevModPhys.61.1}}

@article{Padmanabhan:2002ji,
	archiveprefix = {arXiv},
	author = {Padmanabhan, T.},
	doi = {10.1016/S0370-1573(03)00120-0},
	eprint = {hep-th/0212290},
	journal = {Phys. Rept.},
	pages = {235--320},
	title = {{Cosmological constant: The Weight of the vacuum}},
	volume = {380},
	year = {2003},
	bdsk-url-1 = {https://doi.org/10.1016/S0370-1573(03)00120-0}}

@article{2016arXiv161100036D,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016arXiv161100036D},
	archiveprefix = {arXiv},
	author = {{DESI Collaboration, and others}},
	eid = {arXiv:1611.00036},
	eprint = {1611.00036},
	journal = {arXiv e-prints},
	keywords = {Astrophysics - Instrumentation and Methods for Astrophysics, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	pages = {arXiv:1611.00036},
	primaryclass = {astro-ph.IM},
	title = {{The DESI Experiment Part I: Science,Targeting, and Survey Design}},
	year = 2016}

@article{2016arXiv161100037D,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016arXiv161100037D},
	archiveprefix = {arXiv},
	author = {{DESI Collaboration, and others}},
	eid = {arXiv:1611.00037},
	eprint = {1611.00037},
	journal = {arXiv e-prints},
	keywords = {Astrophysics - Instrumentation and Methods for Astrophysics, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	pages = {arXiv:1611.00037},
	primaryclass = {astro-ph.IM},
	title = {{The DESI Experiment Part II: Instrument Design}},
	year = 2016}

@article{2009arXiv0912.0201L,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2009arXiv0912.0201L},
	archiveprefix = {arXiv},
	author = {{LSST Science Collaboration}},
	eid = {arXiv:0912.0201},
	eprint = {0912.0201},
	journal = {arXiv e-prints},
	keywords = {Astrophysics - Instrumentation and Methods for Astrophysics, Astrophysics - Cosmology and Extragalactic Astrophysics, Astrophysics - Earth and Planetary Astrophysics, Astrophysics - Galaxy Astrophysics, Astrophysics - Solar and Stellar Astrophysics},
	month = dec,
	pages = {arXiv:0912.0201},
	primaryclass = {astro-ph.IM},
	title = {{LSST Science Book, Version 2.0}},
	year = 2009}

@article{Euclid,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2011arXiv1110.3193L},
	archiveprefix = {arXiv},
	author = {{Laureijs}, R. and others},
	eid = {arXiv:1110.3193},
	eprint = {1110.3193},
	journal = {arXiv e-prints},
	keywords = {Astrophysics - Cosmology and Extragalactic Astrophysics, Astrophysics - Galaxy Astrophysics},
	month = oct,
	pages = {arXiv:1110.3193},
	primaryclass = {astro-ph.CO},
	title = {{Euclid Definition Study Report}},
	year = 2011}

@article{2013JCAP...06..036T,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2013JCAP...06..036T},
	archiveprefix = {arXiv},
	author = {{Tassev}, Svetlin and {Zaldarriaga}, Matias and {Eisenstein}, Daniel J.},
	doi = {10.1088/1475-7516/2013/06/036},
	eid = {036},
	eprint = {1301.0322},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Extragalactic Astrophysics},
	month = jun,
	number = {6},
	pages = {036},
	primaryclass = {astro-ph.CO},
	title = {{Solving large scale structure in ten easy steps with COLA}},
	volume = {2013},
	year = 2013,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2013/06/036}}


%and {Delubac}, T. and {Rich}, J.
@ARTICLE{ly-alpha_BOSS,
       author = {{Busca}, N.~G.  and {et al.}},
        title = "{Baryon acoustic oscillations in the Ly{\ensuremath{$\alpha$}} forest of BOSS quasars}",
      journal = {\aap},
     keywords = {cosmology: observations, dark energy, large-scale structure of Universe, cosmological parameters, Astrophysics - Cosmology and Extragalactic Astrophysics},
         year = 2013,
        month = apr,
       volume = {552},
          eid = {A96},
        pages = {A96},
          doi = {10.1051/0004-6361/201220724},
archivePrefix = {arXiv},
       eprint = {1211.2616},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2013A&A...552A..96B},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@article{ly-alpha_BOSS2,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2013JCAP...04..026S},
	archiveprefix = {arXiv},
	author = {{Slosar}, An{\v{z}}e and others},
	doi = {10.1088/1475-7516/2013/04/026},
	eid = {026},
	eprint = {1301.3459},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, High Energy Physics - Experiment},
	month = apr,
	number = {4},
	pages = {026},
	primaryclass = {astro-ph.CO},
	title = {{Measurement of baryon acoustic oscillations in the Lyman-{\ensuremath{\alpha}} forest fluctuations in BOSS data release 9}},
	volume = {2013},
	year = 2013,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2013/04/026}}

@article{Ly-alpha_cross,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2014JCAP...05..027F},
	archiveprefix = {arXiv},
	author = {{Font-Ribera}, Andreu and others},
	doi = {10.1088/1475-7516/2014/05/027},
	eid = {027},
	eprint = {1311.1767},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = may,
	number = {5},
	pages = {027},
	primaryclass = {astro-ph.CO},
	title = {{Quasar-Lyman {\ensuremath{\alpha}} forest cross-correlation from BOSS DR11: Baryon Acoustic Oscillations}},
	volume = {2014},
	year = 2014,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2014/05/027}}


@ARTICLE{ly-alpha_eBOSS,
       author = {{du Mas des Bourboux}, H{\'e}lion and {et al.}},
        title = "{The Completed SDSS-IV Extended Baryon Oscillation Spectroscopic Survey: Baryon Acoustic Oscillations with Ly{\ensuremath{\alpha}} Forests}",
      journal = {\apj},
     keywords = {Dark energy, Cosmology, Large-scale structure of the universe, Baryon acoustic oscillations, Ly{\ensuremath{\alpha}} forest, 351, 343, 902, 138, 980, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2020,
        month = oct,
       volume = {901},
       number = {2},
          eid = {153},
        pages = {153},
          doi = {10.3847/1538-4357/abb085},
archivePrefix = {arXiv},
       eprint = {2007.08995},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2020ApJ...901..153D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




@article{ozdes:2017,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2017MNRAS.472..273C},
	archiveprefix = {arXiv},
	author = {{Childress}, M.~J. and {Lidman}, C. and {Davis}, T.~M. and {Tucker}, B.~E. and {Asorey}, J. and {Yuan}, F. and {Abbott}, T.~M.~C. and {Abdalla}, F.~B. and {Allam}, S. and {Annis}, J. and others},
	doi = {10.1093/mnras/stx1872},
	eprint = {1708.04526},
	journal = {\mnras},
	keywords = {supernovae: general, dark energy, supernovae: general - dark energy, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies},
	month = nov,
	number = {1},
	pages = {273-288},
	primaryclass = {astro-ph.CO},
	title = {{OzDES multifibre spectroscopy for the Dark Energy Survey: 3-yr results and first data release}},
	volume = {472},
	year = 2017,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stx1872}}

@article{2019ApJ...872L..30A,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2019ApJ...872L..30A},
	archiveprefix = {arXiv},
	author = {Dark Energy Survey Collaboration},
	doi = {10.3847/2041-8213/ab04fa},
	eid = {L30},
	eprint = {1811.02374},
	journal = {\apjl},
	keywords = {dark energy, dark matter, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = feb,
	number = {2},
	pages = {L30},
	primaryclass = {astro-ph.CO},
	title = {{First Cosmology Results using Type Ia Supernovae from the Dark Energy Survey: Constraints on Cosmological Parameters}},
	volume = {872},
	year = 2019,
	bdsk-url-1 = {https://doi.org/10.3847/2041-8213/ab04fa}}

@article{sdssdr14,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018ApJS..235...42A},
	archiveprefix = {arXiv},
	author = {{SDSS Collaboration,}},
	doi = {10.3847/1538-4365/aa9e8a},
	eid = {42},
	eprint = {1707.09322},
	journal = {\apjs},
	keywords = {atlases, catalogs, surveys, Astrophysics - Astrophysics of Galaxies, Astrophysics - Instrumentation and Methods for Astrophysics},
	month = apr,
	number = {2},
	pages = {42},
	primaryclass = {astro-ph.GA},
	title = {{The Fourteenth Data Release of the Sloan Digital Sky Survey: First Spectroscopic Data from the Extended Baryon Oscillation Spectroscopic Survey and from the Second Phase of the Apache Point Observatory Galactic Evolution Experiment}},
	volume = {235},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.3847/1538-4365/aa9e8a}}

@article{2018ApJS..239...18A,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018ApJS..239...18A},
	archiveprefix = {arXiv},
	author = {{Dark Energy Survey Collaboration}},
	doi = {10.3847/1538-4365/aae9f0},
	eid = {18},
	eprint = {1801.03181},
	journal = {\apjs},
	keywords = {astronomical databases: miscellaneous, catalogs, cosmology: observations, surveys, techniques: image processing, techniques: photometric, Astrophysics - Instrumentation and Methods for Astrophysics, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies, Astrophysics - Solar and Stellar Astrophysics},
	month = dec,
	number = {2},
	pages = {18},
	primaryclass = {astro-ph.IM},
	title = {{The Dark Energy Survey: Data Release 1}},
	volume = {239},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.3847/1538-4365/aae9f0}}

@ARTICLE{Riquelme23,
       author = {{Riquelme}, Walter and DES},
        title = "{Primordial non-Gaussianity with angular correlation function: integral constraint and validation for DES}",
      journal = {\mnras},
     keywords = {cosmology: observations, inflation, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2023,
        month = jul,
       volume = {523},
       number = {1},
        pages = {603-619},
          doi = {10.1093/mnras/stad1429},
archivePrefix = {arXiv},
       eprint = {2209.07187},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2023MNRAS.523..603R},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@article{elvinpoole,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018PhRvD..98d2006E},
	archiveprefix = {arXiv},
	author = {{Elvin-Poole}, J. and DES},
	doi = {10.1103/PhysRevD.98.042006},
	eid = {042006},
	eprint = {1708.01536},
	journal = {\prd},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = {Aug},
	number = {4},
	pages = {042006},
	primaryclass = {astro-ph.CO},
	title = {{Dark Energy Survey year 1 results: Galaxy clustering for combined probes}},
	volume = {98},
	year = {2018},
	bdsk-url-1 = {https://doi.org/10.1103/PhysRevD.98.042006}}

@article{Chan:2018gtc,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018MNRAS.480.3031C},
	archiveprefix = {arXiv},
	author = {{Chan}, K.~C. and DES},
	doi = {10.1093/mnras/sty2036},
	eprint = {1801.04390},
	journal = {\mnras},
	keywords = {cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = nov,
	number = {3},
	pages = {3031-3051},
	primaryclass = {astro-ph.CO},
	title = {{BAO from angular clustering: optimization and mitigation of theoretical systematics}},
	volume = {480},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/sty2036}}

@article{DESY1baosample,
	archiveprefix = {arXiv},
	author = {Crocce, M. and DES},
	collaboration = {DES},
	doi = {10.1093/mnras/sty2522},
	eprint = {1712.06211},
	journal = {\mnras},
	number = {2},
	pages = {2807--2822},
	primaryclass = {astro-ph.CO},
	reportnumber = {FERMILAB-PUB-17-585-A-AD-AE-SCD},
	title = {{Dark Energy Survey Year 1 Results: Galaxy Sample for BAO Measurement}},
	volume = {482},
	year = {2019},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/sty2522}}

@article{DESY1baomocks,
	archiveprefix = {arXiv},
	author = {{Avila}, S. and DES},
	doi = {10.1093/mnras/sty1389},
	eprint = {1712.06232},
	journal = {\mnras},
	number = {1},
	pages = {94--110},
	primaryclass = {astro-ph.CO},
	reportnumber = {FERMILAB-PUB-17-587-A-AD-AE-SCD, IFT-UAM-CSIC-17-124, DES-2017-0292},
	title = {{Dark Energy Survey Year 1 Results: galaxy mock catalogues for BAO}},
	volume = {479},
	year = {2018},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/sty1389}}

@article{Ross:2017emc,
	archiveprefix = {arXiv},
	author = {Ross, Ashley J. and DES},
	doi = {10.1093/mnras/stx2120},
	eprint = {1705.05442},
	journal = {\mnras},
	number = {4},
	pages = {4456--4468},
	primaryclass = {astro-ph.CO},
	reportnumber = {FERMILAB-PUB-17-162-A-AE, IFT-UAM-CSIC-17-044},
	title = {{Optimized Clustering Estimators for BAO Measurements Accounting for Significant Redshift Uncertainty}},
	volume = {472},
	year = {2017},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stx2120}}

@article{2018MNRAS.480.3031C,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018MNRAS.480.3031C},
	archiveprefix = {arXiv},
	author = {{Chan}, K.~C. and DES},
	doi = {10.1093/mnras/sty2036},
	eprint = {1801.04390},
	journal = {\mnras},
	keywords = {cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = nov,
	number = {3},
	pages = {3031-3051},
	primaryclass = {astro-ph.CO},
	title = {{BAO from angular clustering: optimization and mitigation of theoretical systematics}},
	volume = {480},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/sty2036}}

@article{Cuceu_etal2020,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020JCAP...07..035C},
	archiveprefix = {arXiv},
	author = {{Cuceu}, Andrei and {Font-Ribera}, Andreu and {Joachimi}, Benjamin},
	doi = {10.1088/1475-7516/2020/07/035},
	eid = {035},
	eprint = {2004.02761},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jul,
	number = {7},
	pages = {035},
	primaryclass = {astro-ph.CO},
	title = {{Bayesian methods for fitting Baryon Acoustic Oscillations in the Lyman-{\ensuremath{\alpha}} forest}},
	volume = {2020},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2020/07/035}}

@article{Chan21,
	author = {{Chan}, Kwan Chuen and {Ferrero}, Ismael and {Avila}, Santiago and {Ross}, Ashley J. and {Crocce}, Martin and {Gaztanaga}, Enrique},
        title = "{Clustering with general photo-z uncertainties: application to Baryon Acoustic Oscillations}",
      journal = {\mnras},
     keywords = {cosmology: observations, (cosmology:) large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = apr,
       volume = {511},
       number = {3},
        pages = {3965-3982},
          doi = {10.1093/mnras/stac340},
archivePrefix = {arXiv},
       eprint = {2110.13332},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022MNRAS.511.3965C},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{2020JCAP...05..042I,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020JCAP...05..042I},
	archiveprefix = {arXiv},
	author = {{Ivanov}, Mikhail M. and {Simonovi{\'c}}, Marko and {Zaldarriaga}, Matias},
	doi = {10.1088/1475-7516/2020/05/042},
	eid = {042},
	eprint = {1909.05277},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, General Relativity and Quantum Cosmology, High Energy Physics - Phenomenology},
	month = may,
	number = {5},
	pages = {042},
	primaryclass = {astro-ph.CO},
	title = {{Cosmological parameters from the BOSS galaxy power spectrum}},
	volume = {2020},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2020/05/042}}

@article{2018JCAP...07..053I,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018JCAP...07..053I},
	archiveprefix = {arXiv},
	author = {{Ivanov}, Mikhail M. and {Sibiryakov}, Sergey},
	doi = {10.1088/1475-7516/2018/07/053},
	eid = {053},
	eprint = {1804.05080},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, High Energy Physics - Phenomenology, High Energy Physics - Theory},
	month = jul,
	number = {7},
	pages = {053},
	primaryclass = {astro-ph.CO},
	title = {{Infrared resummation for biased tracers in redshift space}},
	volume = {2018},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2018/07/053}}

@article{2016JCAP...07..028B,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016JCAP...07..028B},
	archiveprefix = {arXiv},
	author = {{Blas}, Diego and {Garny}, Mathias and {Ivanov}, Mikhail M. and {Sibiryakov}, Sergey},
	doi = {10.1088/1475-7516/2016/07/028},
	eid = {028},
	eprint = {1605.02149},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, High Energy Physics - Phenomenology, High Energy Physics - Theory},
	month = jul,
	number = {7},
	pages = {028},
	primaryclass = {astro-ph.CO},
	title = {{Time-sliced perturbation theory II: baryon acoustic oscillations and infrared resummation}},
	volume = {2016},
	year = 2016,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2016/07/028}}

@article{Camacho:2018mel,
	archiveprefix = {arXiv},
	author = {Camacho, H. and DES},
	doi = {10.1093/mnras/stz1514},
	eprint = {1807.10163},
	journal = {\mnras},
	number = {3},
	pages = {3870--3883},
	primaryclass = {astro-ph.CO},
	reportnumber = {FERMILAB-PUB-18-387-AE-CD},
	title = {{Dark Energy Survey Year 1 Results: Measurement of the Galaxy Angular Power Spectrum}},
	volume = {487},
	year = {2019},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stz1514}}

@article{2015JCAP...02..013S,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015JCAP...02..013S},
	archiveprefix = {arXiv},
	author = {{Senatore}, Leonardo and {Zaldarriaga}, Matias},
	doi = {10.1088/1475-7516/2015/02/013},
	eid = {013},
	eprint = {1404.5954},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, General Relativity and Quantum Cosmology, High Energy Physics - Phenomenology, High Energy Physics - Theory},
	month = feb,
	number = {2},
	pages = {013},
	primaryclass = {astro-ph.CO},
	title = {{The IR-resummed Effective Field Theory of Large Scale Structures}},
	volume = {2015},
	year = 2015,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2015/02/013}}


@article{2018MNRAS.479...94A,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018MNRAS.479...94A},
	archiveprefix = {arXiv},
	author = {{Avila}, S. and DES},
	doi = {10.1093/mnras/sty1389},
	eprint = {1712.06232},
	journal = {\mnras},
	keywords = {large-scale structure of Universe, cosmology: theory, cosmology: observations, methods: numerical, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = sep,
	number = {1},
	pages = {94-110},
	primaryclass = {astro-ph.CO},
	title = {{Dark Energy Survey Year-1 results: galaxy mock catalogues for BAO}},
	volume = {479},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/sty1389}}

@article{2015MNRAS.447..646C,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015MNRAS.447..646C},
	archiveprefix = {arXiv},
	author = {{Carretero}, J. and {Castander}, F.~J. and {Gazta{\~n}aga}, E. and {Crocce}, M. and {Fosalba}, P.},
	doi = {10.1093/mnras/stu2402},
	eprint = {1411.3286},
	journal = {\mnras},
	keywords = {Astronomical data bases, catalogues, galaxies: evolution, galaxies: haloes, large-scale structure of Universe, Astrophysics - Astrophysics of Galaxies, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = feb,
	number = {1},
	pages = {646-670},
	primaryclass = {astro-ph.GA},
	title = {{An algorithm to build mock galaxy catalogues using MICE simulations}},
	volume = {447},
	year = 2015,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stu2402}}

@article{Ivanov:2018gjr,
	archiveprefix = {arXiv},
	author = {Ivanov, Mikhail M. and Sibiryakov, Sergey},
	doi = {10.1088/1475-7516/2018/07/053},
	eprint = {1804.05080},
	journal = {\jcap},
	pages = {053},
	primaryclass = {astro-ph.CO},
	reportnumber = {CERN-TH-2018-076, INR-TH-2018-006},
	title = {{Infrared Resummation for Biased Tracers in Redshift Space}},
	volume = {07},
	year = {2018},
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2018/07/053}}

@article{ross11,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/#abs/2011MNRAS.417.1350R},
	archiveprefix = {arXiv},
	author = {{Ross}, Ashley J. and others},
	doi = {10.1111/j.1365-2966.2011.19351.x},
	eprint = {1105.2320},
	journal = {\mnras},
	keywords = {methods: statistical, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = Oct,
	pages = {1350-1373},
	primaryclass = {astro-ph.CO},
	title = {{Ameliorating systematic uncertainties in the angular clustering of galaxies: a study using the SDSS-III}},
	volume = {417},
	year = 2011,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2011.19351.x}}

@article{ice-cola,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.459.2327I},
	archiveprefix = {arXiv},
	author = {{Izard}, Albert and {Crocce}, Martin and {Fosalba}, Pablo},
	doi = {10.1093/mnras/stw797},
	eprint = {1509.04685},
	journal = {\mnras},
	keywords = {methods: numerical, dark matter, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = {Jul},
	number = {3},
	pages = {2327-2341},
	primaryclass = {astro-ph.CO},
	title = {{ICE-COLA: towards fast and accurate synthetic galaxy catalogues optimizing a quasi-N-body method}},
	volume = {459},
	year = {2016},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw797}}

@article{cola,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.459.2118K},
	archiveprefix = {arXiv},
	author = {{Koda}, Jun and {Blake}, Chris and {Beutler}, Florian and {Kazin}, Eyal and {Marin}, Felipe},
	doi = {10.1093/mnras/stw763},
	eprint = {1507.05329},
	journal = {\mnras},
	keywords = {methods: numerical, cosmology: theory, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = {Jun},
	number = {2},
	pages = {2118-2129},
	primaryclass = {astro-ph.CO},
	title = {{Fast and accurate mock catalogue generation for low-mass galaxies}},
	volume = {459},
	year = {2016},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw763}}

@ARTICLE{halogen,
       author = {{Avila}, Santiago and {Murray}, Steven G. and {Knebe}, Alexander and {Power}, Chris and {Robotham}, Aaron S.~G. and {Garcia-Bellido}, Juan},
        title = "{HALOGEN: a tool for fast generation of mock halo catalogues}",
      journal = {\mnras},
     keywords = {galaxies: distances and redshifts, galaxies: haloes, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2015,
        month = jun,
       volume = {450},
       number = {2},
        pages = {1856-1867},
          doi = {10.1093/mnras/stv711},
archivePrefix = {arXiv},
       eprint = {1412.5228},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2015MNRAS.450.1856A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@article{MICE1,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015MNRAS.448.2987F},
	archiveprefix = {arXiv},
	author = {{Fosalba}, P. and {Crocce}, M. and {Gazta{\~n}aga}, E. and {Castand er}, F.~J.},
	doi = {10.1093/mnras/stv138},
	eprint = {1312.1707},
	journal = {\mnras},
	keywords = {methods: numerical, galaxies: statistics, dark matter, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = {Apr},
	number = {4},
	pages = {2987-3000},
	primaryclass = {astro-ph.CO},
	title = {{The MICE grand challenge lightcone simulation - I. Dark matter clustering}},
	volume = {448},
	year = {2015},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stv138}}

@article{MICE2,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015MNRAS.453.1513C},
	archiveprefix = {arXiv},
	author = {{Crocce}, M. and {Castander}, F.~J. and {Gazta{\~n}aga}, E. and {Fosalba}, P. and {Carretero}, J.},
	doi = {10.1093/mnras/stv1708},
	eprint = {1312.2013},
	journal = {\mnras},
	keywords = {methods: numerical, cosmology: observations, dark energy, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = {Oct},
	number = {2},
	pages = {1513-1530},
	primaryclass = {astro-ph.CO},
	title = {{The MICE Grand Challenge lightcone simulation - II. Halo and galaxy catalogues}},
	volume = {453},
	year = {2015},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stv1708}}

@article{MICE3,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015MNRAS.447.1319F},
	archiveprefix = {arXiv},
	author = {{Fosalba}, P. and {Gazta{\~n}aga}, E. and {Castander}, F.~J. and {Crocce}, M.},
	doi = {10.1093/mnras/stu2464},
	eprint = {1312.2947},
	journal = {\mnras},
	keywords = {gravitational lensing: weak, methods: analytical, methods: numerical, galaxies: general, cosmology: theory, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = feb,
	number = {2},
	pages = {1319-1332},
	primaryclass = {astro-ph.CO},
	title = {{The MICE Grand Challenge light-cone simulation - III. Galaxy lensing mocks from all-sky lensing maps}},
	volume = {447},
	year = 2015,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stu2464}}

@article{Xavier:2016elr,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.459.3693X},
	archiveprefix = {arXiv},
	author = {{Xavier}, Henrique S. and {Abdalla}, Filipe B. and {Joachimi}, Benjamin},
	doi = {10.1093/mnras/stw874},
	eprint = {1602.08503},
	journal = {\mnras},
	keywords = {gravitational lensing: weak, methods: statistical, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jul,
	number = {4},
	pages = {3693-3710},
	primaryclass = {astro-ph.CO},
	title = {{Improving lognormal models for cosmological fields}},
	volume = {459},
	year = 2016,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw874}}

@article{2004MNRAS.352..338J,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2004MNRAS.352..338J},
	archiveprefix = {arXiv},
	author = {{Jarvis}, M. and {Bernstein}, G. and {Jain}, B.},
	doi = {10.1111/j.1365-2966.2004.07926.x},
	eprint = {astro-ph/0307393},
	journal = {\mnras},
	keywords = {gravitational lensing, Astrophysics},
	month = jul,
	number = {1},
	pages = {338-352},
	primaryclass = {astro-ph},
	title = {{The skewness of the aperture mass statistic}},
	volume = {352},
	year = 2004,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2004.07926.x}}

@article{treecorr,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015ascl.soft08007J},
	archiveprefix = {ascl},
	author = {{Jarvis}, Mike},
	eid = {ascl:1508.007},
	eprint = {1508.007},
	journal = {Astrophysics Source Code Library},
	keywords = {Software},
	month = aug,
	pages = {ascl:1508.007},
	title = {{TreeCorr: Two-point correlation functions}},
	year = 2015}

@article{cute,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2012arXiv1210.1833A},
	archiveprefix = {arXiv},
	author = {{Alonso}, David},
	eid = {arXiv:1210.1833},
	eprint = {1210.1833},
	journal = {arXiv e-prints},
	keywords = {Astrophysics - Instrumentation and Methods for Astrophysics},
	month = oct,
	pages = {arXiv:1210.1833},
	primaryclass = {astro-ph.IM},
	title = {{CUTE solutions for two-point correlation functions from large cosmological datasets}},
	year = 2012}

@article{Coles:1991if,
	author = {Coles, Peter and Jones, Bernard},
	journal = {\mnras},
	pages = {1--13},
	title = {{A Lognormal model for the cosmological mass distribution}},
	volume = {248},
	year = {1991}}

@article{Clerkin:2016kyr,
	archiveprefix = {arXiv},
	author = {Clerkin, L. and others},
	collaboration = {DES},
	doi = {10.1093/mnras/stw2106},
	eprint = {1605.02036},
	journal = {\mnras},
	number = {2},
	pages = {1444--1461},
	primaryclass = {astro-ph.CO},
	reportnumber = {FERMILAB-PUB-16-161-AE},
	title = {{Testing the lognormality of the galaxy and weak lensing convergence distributions from Dark Energy Survey maps}},
	volume = {466},
	year = {2017},
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw2106}}

@article{2000ApJ...538..473L,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2000ApJ...538..473L},
	archiveprefix = {arXiv},
	author = {{Lewis}, Antony and {Challinor}, Anthony and {Lasenby}, Anthony},
	doi = {10.1086/309179},
	eprint = {astro-ph/9911177},
	journal = {\apj},
	keywords = {Cosmology: Cosmic Microwave Background, Cosmology: Theory, Astrophysics},
	month = aug,
	number = {2},
	pages = {473-476},
	primaryclass = {astro-ph},
	title = {{Efficient Computation of Cosmic Microwave Background Anisotropies in Closed Friedmann-Robertson-Walker Models}},
	volume = {538},
	year = 2000,
	bdsk-url-1 = {https://doi.org/10.1086/309179}}

@article{2016JCAP...03..057V,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2016JCAP...03..057V},
	archiveprefix = {arXiv},
	author = {{Vlah}, Zvonimir and {Seljak}, Uro{\v{s}} and {Yat Chu}, Man and {Feng}, Yu},
	doi = {10.1088/1475-7516/2016/03/057},
	eid = {057},
	eprint = {1509.02120},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = mar,
	number = {3},
	pages = {057},
	primaryclass = {astro-ph.CO},
	title = {{Perturbation theory, effective field theory, and oscillations in the power spectrum}},
	volume = {2016},
	year = 2016,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2016/03/057}}

@article{LS1993,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/1993ApJ...412...64L},
	author = {{Landy}, Stephen D. and {Szalay}, Alexander S.},
	doi = {10.1086/172900},
	journal = {\apj},
	keywords = {Angular Correlation, Bias, Computational Astrophysics, Galactic Clusters, Variance, Correlation Coefficients, Monte Carlo Method, Random Variables, Astrophysics, GALAXIES: CLUSTERING, METHODS: NUMERICAL},
	month = jul,
	pages = {64},
	title = {{Bias and Variance of Angular Correlation Functions}},
	volume = {412},
	year = 1993,
	bdsk-url-1 = {https://doi.org/10.1086/172900}}

@article{1987MNRAS.227....1K,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/1987MNRAS.227....1K},
	author = {{Kaiser}, Nick},
	doi = {10.1093/mnras/227.1.1},
	journal = {\mnras},
	keywords = {Astronomical Models, Deep Space, Galactic Clusters, Red Shift, Density Distribution, Power Spectra, Velocity Distribution, Astrophysics},
	month = jul,
	pages = {1-21},
	title = {{Clustering in real space and in redshift space}},
	volume = {227},
	year = 1987,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/227.1.1}}

@article{FKP,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/1994ApJ...426...23F},
	archiveprefix = {arXiv},
	author = {{Feldman}, Hume A. and {Kaiser}, Nick and {Peacock}, John A.},
	doi = {10.1086/174036},
	eprint = {astro-ph/9304022},
	journal = {\apj},
	keywords = {Astronomical Models, Cosmology, Dark Matter, Normal Density Functions, Power Spectra, Red Shift, Sky Surveys (Astronomy), Three Dimensional Models, Fourier Transformation, Infrared Astronomy Satellite, Stochastic Processes, Weighting Functions, Astrophysics, COSMOLOGY: THEORY, GALAXIES: DISTANCES AND REDSHIFTS, INFRARED: GALAXIES, METHODS: STATISTICAL, SURVEYS, Astrophysics},
	month = may,
	pages = {23},
	primaryclass = {astro-ph},
	title = {{Power-Spectrum Analysis of Three-dimensional Redshift Surveys}},
	volume = {426},
	year = 1994,
	bdsk-url-1 = {https://doi.org/10.1086/174036}}

@article{PadmanabhanWhite_2009,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {http://adsabs.harvard.edu/abs/2009PhRvD..80f3508P},
	archiveprefix = {arXiv},
	author = {{Padmanabhan}, N. and {White}, M.},
	doi = {10.1103/PhysRevD.80.063508},
	eid = {063508},
	eprint = {0906.1198},
	journal = {\prd},
	keywords = {Dark energy, Cosmology},
	month = sep,
	number = 6,
	pages = {063508},
	primaryclass = {astro-ph.CO},
	title = {{Calibrating the baryon oscillation ruler for matter and halos}},
	volume = 80,
	year = 2009,
	bdsk-url-1 = {https://doi.org/10.1103/PhysRevD.80.063508}}

@article{CrocceScoccimarro_2008,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {http://adsabs.harvard.edu/abs/2008PhRvD..77b3533C},
	archiveprefix = {arXiv},
	author = {{Crocce}, M. and {Scoccimarro}, R.},
	doi = {10.1103/PhysRevD.77.023533},
	eid = {023533},
	eprint = {0704.2783},
	journal = {\prd},
	keywords = {Cosmology},
	month = jan,
	number = 2,
	pages = {023533},
	title = {{Nonlinear evolution of baryon acoustic oscillations}},
	volume = 77,
	year = 2008,
	bdsk-url-1 = {https://doi.org/10.1103/PhysRevD.77.023533}}

@article{1998AJ....116.1009R,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/1998AJ....116.1009R},
	archiveprefix = {arXiv},
	author = {{Riess}, Adam G. and others},
	doi = {10.1086/300499},
	eprint = {astro-ph/9805201},
	journal = {\aj},
	keywords = {COSMOLOGY: OBSERVATIONS, STARS: SUPERNOVAE: GENERAL, Astrophysics},
	month = sep,
	number = {3},
	pages = {1009-1038},
	primaryclass = {astro-ph},
	title = {{Observational Evidence from Supernovae for an Accelerating Universe and a Cosmological Constant}},
	volume = {116},
	year = 1998,
	bdsk-url-1 = {https://doi.org/10.1086/300499}}

@article{1999ApJ...517..565P,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/1999ApJ...517..565P},
	archiveprefix = {arXiv},
	author = {{Perlmutter}, S. and others},
	doi = {10.1086/307221},
	eprint = {astro-ph/9812133},
	journal = {\apj},
	keywords = {COSMOLOGY: OBSERVATIONS, COSMOLOGY: DISTANCE SCALE, STARS: SUPERNOVAE: GENERAL, Cosmology: Observations, Cosmology: Distance Scale, Stars: Supernovae: General, Astrophysics, High Energy Physics - Experiment, High Energy Physics - Phenomenology},
	month = jun,
	number = {2},
	pages = {565-586},
	primaryclass = {astro-ph},
	title = {{Measurements of {\ensuremath{\Omega}} and {\ensuremath{\Lambda}} from 42 High-Redshift Supernovae}},
	volume = {517},
	year = 1999,
	bdsk-url-1 = {https://doi.org/10.1086/307221}}

@article{2020PTEP.2020h3C01P,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020PTEP.2020h3C01P},
	author = {{Particle Data Group} and others},
	doi = {10.1093/ptep/ptaa104},
	eid = {083C01},
	journal = {Progress of Theoretical and Experimental Physics},
	month = aug,
	number = {8},
	pages = {083C01},
	title = {{Review of Particle Physics}},
	volume = {2020},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/ptep/ptaa104}}

@article{2005ApJ...633..560E,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2005ApJ...633..560E},
	archiveprefix = {arXiv},
	author = {{Eisenstein}, Daniel J. and others},
	doi = {10.1086/466512},
	eprint = {astro-ph/0501171},
	journal = {\apj},
	keywords = {Cosmology: Cosmic Microwave Background, Cosmology: Cosmological Parameters, Cosmology: Observations, Cosmology: Distance Scale, Galaxies: Elliptical and Lenticular, cD, Cosmology: Large-Scale Structure of Universe, Astrophysics},
	month = nov,
	number = {2},
	pages = {560-574},
	primaryclass = {astro-ph},
	title = {{Detection of the Baryon Acoustic Peak in the Large-Scale Correlation Function of SDSS Luminous Red Galaxies}},
	volume = {633},
	year = 2005,
	bdsk-url-1 = {https://doi.org/10.1086/466512}}

@article{2007MNRAS.381.1053P,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2007MNRAS.381.1053P},
	archiveprefix = {arXiv},
	author = {{Percival}, Will J. and {Cole}, Shaun and {Eisenstein}, Daniel J. and {Nichol}, Robert C. and {Peacock}, John A. and {Pope}, Adrian C. and {Szalay}, Alexander S.},
	doi = {10.1111/j.1365-2966.2007.12268.x},
	eprint = {0705.3323},
	journal = {\mnras},
	keywords = {cosmology: observations, distance scale, large-scale structure of Universe, Astrophysics},
	month = nov,
	number = {3},
	pages = {1053-1066},
	primaryclass = {astro-ph},
	title = {{Measuring the Baryon Acoustic Oscillation scale using the Sloan Digital Sky Survey and 2dF Galaxy Redshift Survey}},
	volume = {381},
	year = 2007,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2007.12268.x}}

@article{2009MNRAS.399.1663G,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {http://adsabs.harvard.edu/abs/2009MNRAS.399.1663G},
	archiveprefix = {arXiv},
	author = {{Gazta{\~n}aga}, E. and {Cabr{\'e}}, A. and {Hui}, L.},
	doi = {10.1111/j.1365-2966.2009.15405.x},
	eprint = {arXiv: 0807.3551 [astro-ph]},
	journal = {\mnras},
	keywords = {cosmology: observations , large-scale structure of Universe},
	month = nov,
	pages = {1663-1680},
	title = {{Clustering of luminous red galaxies - IV. Baryon acoustic peak in the line-of-sight direction and a direct measurement of H(z)}},
	volume = 399,
	year = 2009,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2009.15405.x}}

@article{2010MNRAS.401.2148P,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2010MNRAS.401.2148P},
	archiveprefix = {arXiv},
	author = {{Percival}, Will J. and others},
	doi = {10.1111/j.1365-2966.2009.15812.x},
	eprint = {0907.1660},
	journal = {\mnras},
	keywords = {cosmology: observations, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Extragalactic Astrophysics},
	month = feb,
	number = {4},
	pages = {2148-2168},
	primaryclass = {astro-ph.CO},
	title = {{Baryon acoustic oscillations in the Sloan Digital Sky Survey Data Release 7 galaxy sample}},
	volume = {401},
	year = 2010,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2009.15812.x}}

@article{2015MNRAS.449..835R,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015MNRAS.449..835R},
	archiveprefix = {arXiv},
	author = {{Ross}, Ashley J. and {Samushia}, Lado and {Howlett}, Cullan and {Percival}, Will J. and {Burden}, Angela and {Manera}, Marc},
	doi = {10.1093/mnras/stv154},
	eprint = {1409.3242},
	journal = {\mnras},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = may,
	number = {1},
	pages = {835-847},
	primaryclass = {astro-ph.CO},
	title = {{The clustering of the SDSS DR7 main Galaxy sample - I. A 4 per cent distance measure at z = 0.15}},
	volume = {449},
	year = 2015,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stv154}}

@article{2018MNRAS.473.4773A,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018MNRAS.473.4773A},
	archiveprefix = {arXiv},
	author = {eBOSS Collaboration},
	doi = {10.1093/mnras/stx2630},
	eprint = {1705.06373},
	journal = {\mnras},
	keywords = {cosmology: observations, dark energy, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = feb,
	number = {4},
	pages = {4773-4794},
	primaryclass = {astro-ph.CO},
	title = {{The clustering of the SDSS-IV extended Baryon Oscillation Spectroscopic Survey DR14 quasar sample: first measurement of baryon acoustic oscillations between redshift 0.8 and 2.2}},
	volume = {473},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stx2630}}

@article{2015A&A...574A..59D,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015A&A...574A..59D},
	archiveprefix = {arXiv},
	author = {{Delubac}, Timoth{\'e}e and others},
	doi = {10.1051/0004-6361/201423969},
	eid = {A59},
	eprint = {1404.1801},
	journal = {\aap},
	keywords = {cosmology: observations, dark energy, large-scale structure of Universe, cosmological parameters, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = feb,
	pages = {A59},
	primaryclass = {astro-ph.CO},
	title = {{Baryon acoustic oscillations in the Ly{\ensuremath{\alpha}} forest of BOSS DR11 quasars}},
	volume = {574},
	year = 2015,
	bdsk-url-1 = {https://doi.org/10.1051/0004-6361/201423969}}

@article{2017A&A...603A..12B,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2017A&A...603A..12B},
	archiveprefix = {arXiv},
	author = {{Bautista}, Julian E. and others},
	doi = {10.1051/0004-6361/201730533},
	eid = {A12},
	eprint = {1702.00176},
	journal = {\aap},
	keywords = {cosmological parameters, dark energy, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jun,
	pages = {A12},
	primaryclass = {astro-ph.CO},
	title = {{Measurement of baryon acoustic oscillation correlations at z = 2.3 with SDSS DR12 Ly{\ensuremath{\alpha}}-Forests}},
	volume = {603},
	year = 2017,
	bdsk-url-1 = {https://doi.org/10.1051/0004-6361/201730533}}

@article{2001MNRAS.327.1297P,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2001MNRAS.327.1297P},
	archiveprefix = {arXiv},
	author = {{Percival}, Will J. and others},
	doi = {10.1046/j.1365-8711.2001.04827.x},
	eprint = {astro-ph/0105252},
	journal = {\mnras},
	keywords = {COSMOLOGICAL PARAMETERS, LARGE-SCALE STRUCTURE OF UNIVERSE, Astrophysics},
	month = nov,
	number = {4},
	pages = {1297-1306},
	primaryclass = {astro-ph},
	title = {{The 2dF Galaxy Redshift Survey: the power spectrum and the matter content of the Universe}},
	volume = {327},
	year = 2001,
	bdsk-url-1 = {https://doi.org/10.1046/j.1365-8711.2001.04827.x}}

@article{2005MNRAS.362..505C,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2005MNRAS.362..505C},
	archiveprefix = {arXiv},
	author = {{Cole}, Shaun and others},
	doi = {10.1111/j.1365-2966.2005.09318.x},
	eprint = {astro-ph/0501174},
	journal = {\mnras},
	keywords = {cosmological parameters, large-scale structure of Universe, Astrophysics},
	month = sep,
	number = {2},
	pages = {505-534},
	primaryclass = {astro-ph},
	title = {{The 2dF Galaxy Redshift Survey: power-spectrum analysis of the final data set and cosmological implications}},
	volume = {362},
	year = 2005,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2005.09318.x}}

@article{2011MNRAS.416.3017B,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2011MNRAS.416.3017B},
	archiveprefix = {arXiv},
	author = {{Beutler}, Florian and others},
	doi = {10.1111/j.1365-2966.2011.19250.x},
	eprint = {1106.3366},
	journal = {\mnras},
	keywords = {surveys, cosmology: observations, dark energy, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	number = {4},
	pages = {3017-3032},
	primaryclass = {astro-ph.CO},
	title = {{The 6dF Galaxy Survey: baryon acoustic oscillations and the local Hubble constant}},
	volume = {416},
	year = 2011,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2011.19250.x}}

@article{2011MNRAS.415.2892B,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2011MNRAS.415.2892B},
	archiveprefix = {arXiv},
	author = {{Blake}, Chris and others},
	doi = {10.1111/j.1365-2966.2011.19077.x},
	eprint = {1105.2862},
	journal = {\mnras},
	keywords = {surveys, cosmological parameters, dark energy, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = aug,
	number = {3},
	pages = {2892-2909},
	primaryclass = {astro-ph.CO},
	title = {{The WiggleZ Dark Energy Survey: testing the cosmological model with baryon acoustic oscillations at z= 0.6}},
	volume = {415},
	year = 2011,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2011.19077.x}}

@article{2007MNRAS.378..852P,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2007MNRAS.378..852P},
	archiveprefix = {arXiv},
	author = {{Padmanabhan}, Nikhil and others},
	doi = {10.1111/j.1365-2966.2007.11593.x},
	eprint = {astro-ph/0605302},
	journal = {\mnras},
	keywords = {cosmological parameters, cosmology: observations, distance scale, large-scale structure of Universe, Astrophysics},
	month = jul,
	number = {3},
	pages = {852-872},
	primaryclass = {astro-ph},
	title = {{The clustering of luminous red galaxies in the Sloan Digital Sky Survey imaging data}},
	volume = {378},
	year = 2007,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2007.11593.x}}

@article{2009ApJ...692..265E,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2009ApJ...692..265E},
	archiveprefix = {arXiv},
	author = {{Estrada}, Juan and {Sefusatti}, Emiliano and {Frieman}, Joshua A.},
	doi = {10.1088/0004-637X/692/1/265},
	eprint = {0801.3485},
	journal = {\apj},
	keywords = {cosmology: observations, galaxies: clusters: individual: MaxBCG, large-scale structure of universe, Astrophysics},
	month = feb,
	number = {1},
	pages = {265-282},
	primaryclass = {astro-ph},
	title = {{The Correlation Function of Optically Selected Galaxy Clusters in the Sloan Digital Sky Survey}},
	volume = {692},
	year = 2009,
	bdsk-url-1 = {https://doi.org/10.1088/0004-637X/692/1/265}}

@article{2010MNRAS.401.2477H,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2010MNRAS.401.2477H},
	archiveprefix = {arXiv},
	author = {{H{\"u}tsi}, Gert},
	doi = {10.1111/j.1365-2966.2009.15824.x},
	eprint = {0910.0492},
	journal = {\mnras},
	keywords = {methods: statistical, galaxies: clusters: general, cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = feb,
	number = {4},
	pages = {2477-2489},
	primaryclass = {astro-ph.CO},
	title = {{Power spectrum of the maxBCG sample: detection of acoustic oscillations using galaxy clusters}},
	volume = {401},
	year = 2010,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2009.15824.x}}

@article{2011MNRAS.417.2577C,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2011MNRAS.417.2577C},
	archiveprefix = {arXiv},
	author = {{Crocce}, M. and {Gazta{\~n}aga}, E. and {Cabr{\'e}}, A. and {Carnero}, A. and {S{\'a}nchez}, E.},
	doi = {10.1111/j.1365-2966.2011.19425.x},
	eprint = {1104.5236},
	journal = {\mnras},
	keywords = {methods: data analysis, cosmological parameters, dark energy, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = nov,
	number = {4},
	pages = {2577-2591},
	primaryclass = {astro-ph.CO},
	title = {{Clustering of photometric luminous red galaxies - I. Growth of structure and baryon acoustic feature}},
	volume = {417},
	year = 2011,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2011.19425.x}}

@article{2012ApJ...761...13S,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2012ApJ...761...13S},
	archiveprefix = {arXiv},
	author = {{Seo}, Hee-Jong and others},
	doi = {10.1088/0004-637X/761/1/13},
	eid = {13},
	eprint = {1201.2172},
	journal = {\apj},
	keywords = {distance scale, cosmological parameters, cosmology: observations, large-scale structure of universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = dec,
	number = {1},
	pages = {13},
	primaryclass = {astro-ph.CO},
	title = {{Acoustic Scale from the Angular Power Spectra of SDSS-III DR8 Photometric Luminous Galaxies}},
	volume = {761},
	year = 2012,
	bdsk-url-1 = {https://doi.org/10.1088/0004-637X/761/1/13}}

@article{2011MNRAS.411..277S,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {http://adsabs.harvard.edu/abs/2011MNRAS.411..277S},
	archiveprefix = {arXiv},
	author = {{S{\'a}nchez}, E. and others},
	doi = {10.1111/j.1365-2966.2010.17679.x},
	eprint = {1006.3226},
	journal = {\mnras},
	keywords = {methods: data analysis, cosmological parameters, dark energy, large-scale structure of Universe},
	month = feb,
	pages = {277-288},
	primaryclass = {astro-ph.CO},
	title = {{Tracing the sound horizon scale with photometric redshift surveys}},
	volume = 411,
	year = 2011,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2010.17679.x}}

@article{2012MNRAS.419.1689C,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {http://adsabs.harvard.edu/abs/2012MNRAS.419.1689C},
	archiveprefix = {arXiv},
	author = {{Carnero}, A. and {S{\'a}nchez}, E. and {Crocce}, M. and {Cabr{\'e}}, A. and {Gazta{\~n}aga}, E.},
	doi = {10.1111/j.1365-2966.2011.19832.x},
	eprint = {1104.5426},
	journal = {\mnras},
	keywords = {methods: data analysis, cosmological parameters, dark energy, large-scale structure of Universe},
	month = jan,
	pages = {1689-1694},
	primaryclass = {astro-ph.CO},
	title = {{Clustering of photometric luminous red galaxies - II. Cosmological implications from the baryon acoustic scale}},
	volume = 419,
	year = 2012,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2011.19832.x}}

@article{2005IJMPA..20.3121F,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2005IJMPA..20.3121F},
	author = {{Flaugher}, Brenna},
	doi = {10.1142/S0217751X05025917},
	journal = {International Journal of Modern Physics A},
	keywords = {Dark energy, galaxies, supernovae},
	month = jan,
	number = {14},
	pages = {3121-3123},
	title = {{The Dark Energy Survey}},
	volume = {20},
	year = 2005,
	bdsk-url-1 = {https://doi.org/10.1142/S0217751X05025917}}

@article{Y1_BAO,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2019MNRAS.483.4866A},
	archiveprefix = {arXiv},
	author = {{Dark Energy Survey Collaboration}},
	doi = {10.1093/mnras/sty3351},
	eprint = {1712.06209},
	journal = {\mnras},
	keywords = {cosmology: observations, (cosmology:) large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = mar,
	number = {4},
	pages = {4866-4883},
	primaryclass = {astro-ph.CO},
	title = {{Dark Energy Survey Year 1 Results: Measurement of the Baryon Acoustic Oscillation scale in the distribution of galaxies to redshift 1}},
	volume = {483},
	year = 2019,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/sty3351}}

@article{Planck,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020A&A...641A...6P},
	archiveprefix = {arXiv},
	author = {{Planck Collaboration}},
	doi = {10.1051/0004-6361/201833910},
	eid = {A6},
	eprint = {1807.06209},
	journal = {\aap},
	keywords = {cosmic background radiation, cosmological parameters, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = sep,
	pages = {A6},
	primaryclass = {astro-ph.CO},
	title = {{Planck 2018 results. VI. Cosmological parameters}},
	volume = {641},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1051/0004-6361/201833910}}



@article{2015MNRAS.453.1513C,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2015MNRAS.453.1513C},
	archiveprefix = {arXiv},
	author = {{Crocce}, M. and {Castander}, F.~J. and {Gazta{\~n}aga}, E. and {Fosalba}, P. and {Carretero}, J.},
	doi = {10.1093/mnras/stv1708},
	eprint = {1312.2013},
	journal = {\mnras},
	keywords = {methods: numerical, cosmology: observations, dark energy, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	number = {2},
	pages = {1513-1530},
	primaryclass = {astro-ph.CO},
	title = {{The MICE Grand Challenge lightcone simulation - II. Halo and galaxy catalogues}},
	volume = {453},
	year = 2015,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stv1708}}

@article{DNF,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {http://adsabs.harvard.edu/abs/2016MNRAS.459.3078D},
	archiveprefix = {arXiv},
	author = {{De Vicente}, J. and {S{\'a}nchez}, E. and {Sevilla-Noarbe}, I.},
	doi = {10.1093/mnras/stw857},
	eprint = {1511.07623},
	journal = {\mnras},
	keywords = {methods: data analysis, surveys, galaxies: distances and redshifts, galaxies: statistics, large-scale structure of Universe},
	month = jul,
	pages = {3078-3088},
	title = {{DNF - Galaxy photometric redshift by Directional Neighbourhood Fitting}},
	volume = 459,
	year = 2016,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw857}}

@article{2002ApJ...567....2H,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2002ApJ...567....2H},
	archiveprefix = {arXiv},
	author = {{Hivon}, Eric and {G{\'o}rski}, Krzysztof M. and {Netterfield}, C. Barth and {Crill}, Brendan P. and {Prunet}, Simon and {Hansen}, Frode},
	doi = {10.1086/338126},
	eprint = {astro-ph/0105302},
	journal = {\apj},
	keywords = {Cosmology: Cosmic Microwave Background, Methods: Statistical, Astrophysics},
	month = mar,
	number = {1},
	pages = {2-17},
	primaryclass = {astro-ph},
	title = {{MASTER of the Cosmic Microwave Background Anisotropy Power Spectrum: A Fast Method for Statistical Analysis of Large and Complex Cosmic Microwave Background Data Sets}},
	volume = {567},
	year = 2002,
	bdsk-url-1 = {https://doi.org/10.1086/338126}}

@article{2019MNRAS.484.4127A,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.4127A},
	archiveprefix = {arXiv},
	author = {{Alonso}, David and {Sanchez}, Javier and {Slosar}, An{\v{z}}e and {LSST Dark Energy Science Collaboration}},
	doi = {10.1093/mnras/stz093},
	eprint = {1809.09603},
	journal = {\mnras},
	keywords = {methods: data analysis, methods: numerical, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = apr,
	number = {3},
	pages = {4127-4151},
	primaryclass = {astro-ph.CO},
	title = {{A unified pseudo-C$_{{\ensuremath{\ell}}}$ framework}},
	volume = {484},
	year = 2019,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stz093}}

@article{2020JCAP...03..044N,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020JCAP...03..044N},
	archiveprefix = {arXiv},
	author = {{Nicola}, Andrina and others},
	doi = {10.1088/1475-7516/2020/03/044},
	eid = {044},
	eprint = {1912.08209},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = mar,
	number = {3},
	pages = {044},
	primaryclass = {astro-ph.CO},
	title = {{Tomographic galaxy clustering with the Subaru Hyper Suprime-Cam first year public data release}},
	volume = {2020},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2020/03/044}}

@article{2019JCAP...11..043G,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2019JCAP...11..043G},
	archiveprefix = {arXiv},
	author = {{Garc{\'\i}a-Garc{\'\i}a}, Carlos and {Alonso}, David and {Bellini}, Emilio},
	doi = {10.1088/1475-7516/2019/11/043},
	eid = {043},
	eprint = {1906.11765},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Instrumentation and Methods for Astrophysics},
	month = nov,
	number = {11},
	pages = {043},
	primaryclass = {astro-ph.CO},
	title = {{Disconnected pseudo-C$_{l}$ covariances for projected large-scale structure data}},
	volume = {2019},
	year = 2019,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2019/11/043}}

@article{2004MNRAS.349..603E,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2004MNRAS.349..603E},
	archiveprefix = {arXiv},
	author = {{Efstathiou}, G.},
	doi = {10.1111/j.1365-2966.2004.07530.x},
	eprint = {astro-ph/0307515},
	journal = {\mnras},
	keywords = {methods: data analysis, methods: statistical, cosmic microwave background, large-scale structure of Universe, Astrophysics},
	month = apr,
	number = {2},
	pages = {603-626},
	primaryclass = {astro-ph},
	title = {{Myths and truths concerning estimation of power spectra: the case for a hybrid estimator}},
	volume = {349},
	year = 2004,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2004.07530.x}}

@article{vipers_pdr2,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018A&A...609A..84S},
	archiveprefix = {arXiv},
	author = {{Scodeggio}, M. and others},
	doi = {10.1051/0004-6361/201630114},
	eid = {A84},
	eprint = {1611.07048},
	journal = {\aap},
	keywords = {cosmology: observations, large-scale structure of Universe, galaxies: distances and redshifts, galaxies: statistics, surveys, Astrophysics - Astrophysics of Galaxies},
	month = {Jan},
	pages = {A84},
	primaryclass = {astro-ph.GA},
	title = {{The VIMOS Public Extragalactic Redshift Survey (VIPERS). Full spectroscopic data and auxiliary information release (PDR-2)}},
	volume = {609},
	year = {2018},
	bdsk-url-1 = {https://doi.org/10.1051/0004-6361/201630114}}

@article{vipers,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2014A&A...566A.108G},
	archiveprefix = {arXiv},
	author = {{Guzzo}, L. and others},
	doi = {10.1051/0004-6361/201321489},
	eid = {A108},
	eprint = {1303.2623},
	journal = {\aap},
	keywords = {cosmology: observations, large-scale structure of Universe, galaxies: distances and redshifts, galaxies: statistics, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = {Jun},
	pages = {A108},
	primaryclass = {astro-ph.CO},
	title = {{The VIMOS Public Extragalactic Redshift Survey (VIPERS). An unprecedented view of galaxies and large-scale structure at 0.5 \&lt; z \&lt; 1.2}},
	volume = {566},
	year = {2014},
	bdsk-url-1 = {https://doi.org/10.1051/0004-6361/201321489}}

@article{Crocce2011,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2011MNRAS.414..329C},
	archiveprefix = {arXiv},
	author = {{Crocce}, Mart{\'\i}n and {Cabr{\'e}}, Anna and {Gazta{\~n}aga}, Enrique},
	doi = {10.1111/j.1365-2966.2011.18393.x},
	eprint = {1004.4640},
	journal = {\mnras},
	keywords = {surveys, galaxies: statistics, dark energy, cosmology: miscellaneous, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jun,
	number = {1},
	pages = {329-349},
	primaryclass = {astro-ph.CO},
	title = {{Modelling the angular correlation function and its full covariance in photometric galaxy surveys}},
	volume = {414},
	year = 2011,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2011.18393.x}}

@article{cosmolike,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2017MNRAS.470.2100K},
	archiveprefix = {arXiv},
	author = {{Krause}, Elisabeth and {Eifler}, Tim},
	doi = {10.1093/mnras/stx1261},
	eprint = {1601.05779},
	journal = {\mnras},
	keywords = {cosmological parameters, large-scale structure of Universe, cosmology: theory, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = sep,
	number = {2},
	pages = {2100-2112},
	primaryclass = {astro-ph.CO},
	title = {{cosmolike - cosmological likelihood analyses for photometric galaxy surveys}},
	volume = {470},
	year = 2017,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stx1261}}

@article{Krause21,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2021arXiv210513548K},
	archiveprefix = {arXiv},
	author = {{Krause}, E. and others},
	eid = {arXiv:2105.13548},
	eprint = {2105.13548},
	journal = {arXiv e-prints},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = may,
	pages = {arXiv:2105.13548},
	primaryclass = {astro-ph.CO},
	title = {{Dark Energy Survey Year 3 Results: Multi-Probe Modeling Strategy and Validation}},
	year = 2021}

@article{Leistedt14,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2014MNRAS.444....2L},
	archiveprefix = {arXiv},
	author = {{Leistedt}, Boris and {Peiris}, Hiranya V.},
	doi = {10.1093/mnras/stu1439},
	eprint = {1404.6530},
	journal = {\mnras},
	keywords = {techniques: photometric, quasars: general, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	number = {1},
	pages = {2-14},
	primaryclass = {astro-ph.CO},
	title = {{Exploiting the full potential of photometric quasar surveys: optimal power spectra through blind mitigation of systematics}},
	volume = {444},
	year = 2014,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stu1439}}

@article{cosmolike2020,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.497.2699F},
	archiveprefix = {arXiv},
	author = {{Fang}, Xiao and {Eifler}, Tim and {Krause}, Elisabeth},
	doi = {10.1093/mnras/staa1726},
	eprint = {2004.04833},
	journal = {\mnras},
	keywords = {cosmological parameters, dark energy, large-scale structure of Universe, cosmology: theory, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies, Astrophysics - Instrumentation and Methods for Astrophysics},
	month = jun,
	number = {3},
	pages = {2699-2714},
	primaryclass = {astro-ph.CO},
	title = {{2D-FFTLog: efficient computation of real-space covariance matrices for galaxy clustering and weak lensing}},
	volume = {497},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa1726}}

@article{cosmolike_curvedsky,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020JCAP...05..010F},
	archiveprefix = {arXiv},
	author = {{Fang}, Xiao and {Krause}, Elisabeth and {Eifler}, Tim and {MacCrann}, Niall},
	doi = {10.1088/1475-7516/2020/05/010},
	eid = {010},
	eprint = {1911.11947},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies, Astrophysics - Instrumentation and Methods for Astrophysics},
	month = may,
	number = {5},
	pages = {010},
	primaryclass = {astro-ph.CO},
	title = {{Beyond Limber: efficient computation of angular power spectra for galaxy clustering and weak lensing}},
	volume = {2020},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2020/05/010}}

@article{cosmolike_mask,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018MNRAS.479.4998T},
	archiveprefix = {arXiv},
	author = {{Troxel}, M.~A. and {Krause}, E. and {Chang}, C. and {Eifler}, T.~F. and {Friedrich}, O. and {Gruen}, D. and {MacCrann}, N. and others},
	doi = {10.1093/mnras/sty1889},
	eprint = {1804.10663},
	journal = {\mnras},
	keywords = {gravitational lensing: weak, methods: data analysis, methods: statistical, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	number = {4},
	pages = {4998-5004},
	primaryclass = {astro-ph.CO},
	title = {{Survey geometry and the internal consistency of recent cosmic shear measurements}},
	volume = {479},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/sty1889}}

@article{Cooray2002,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2002PhR...372....1C},
	archiveprefix = {arXiv},
	author = {{Cooray}, Asantha and {Sheth}, Ravi},
	doi = {10.1016/S0370-1573(02)00276-4},
	eprint = {astro-ph/0206508},
	journal = {\physrep},
	keywords = {Astrophysics},
	month = dec,
	number = {1},
	pages = {1-129},
	primaryclass = {astro-ph},
	title = {{Halo models of large scale structure}},
	volume = {372},
	year = 2002,
	bdsk-url-1 = {https://doi.org/10.1016/S0370-1573(02)00276-4}}

@article{Takada2009,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2009MNRAS.395.2065T},
	archiveprefix = {arXiv},
	author = {{Takada}, Masahiro and {Jain}, Bhuvnesh},
	doi = {10.1111/j.1365-2966.2009.14504.x},
	eprint = {0810.4170},
	journal = {\mnras},
	keywords = {gravitational lensing, cosmology: theory, large-scale structure of universe, Astrophysics},
	month = jun,
	number = {4},
	pages = {2065-2086},
	primaryclass = {astro-ph},
	title = {{The impact of non-Gaussian errors on weak lensing surveys}},
	volume = {395},
	year = 2009,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2009.14504.x}}

@article{Takada2013,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2013PhRvD..87l3504T},
	archiveprefix = {arXiv},
	author = {{Takada}, Masahiro and {Hu}, Wayne},
	doi = {10.1103/PhysRevD.87.123504},
	eid = {123504},
	eprint = {1302.6994},
	journal = {\prd},
	keywords = {98.80.-k, 95.36.+x, 98.65.Dx, Cosmology, Dark energy, Superclusters, large-scale structure of the Universe, Astrophysics - Cosmology and Extragalactic Astrophysics},
	month = jun,
	number = {12},
	pages = {123504},
	primaryclass = {astro-ph.CO},
	title = {{Power spectrum super-sample covariance}},
	volume = {87},
	year = 2013,
	bdsk-url-1 = {https://doi.org/10.1103/PhysRevD.87.123504}}

@article{2017MNRAS.470.2617A,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2017MNRAS.470.2617A},
	archiveprefix = {arXiv},
	author = {{BOSS Collaboration}},
	doi = {10.1093/mnras/stx721},
	eprint = {1607.03155},
	journal = {\mnras},
	keywords = {distance scale, large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = sep,
	number = {3},
	pages = {2617-2652},
	primaryclass = {astro-ph.CO},
	title = {{The clustering of galaxies in the completed SDSS-III Baryon Oscillation Spectroscopic Survey: cosmological analysis of the DR12 galaxy sample}},
	volume = {470},
	year = 2017,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stx721}}

@article{eBOSS_final,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2021PhRvD.103h3533A},
	archiveprefix = {arXiv},
	author = {eBOSS Collaboration},
	doi = {10.1103/PhysRevD.103.083533},
	eid = {083533},
	eprint = {2007.08991},
	journal = {\prd},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = apr,
	number = {8},
	pages = {083533},
	primaryclass = {astro-ph.CO},
	title = {{Completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: Cosmological implications from two decades of spectroscopic surveys at the Apache Point Observatory}},
	volume = {103},
	year = 2021,
	bdsk-url-1 = {https://doi.org/10.1103/PhysRevD.103.083533}}

@article{2020MNRAS.500..736B,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.500..736B},
	archiveprefix = {arXiv},
	author = {{Bautista}, Julian E. and others},
	doi = {10.1093/mnras/staa2800},
	eprint = {2007.08993},
	journal = {\mnras},
	keywords = {cosmology: dark energy, cosmology: large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = sep,
	number = {1},
	pages = {736-762},
	primaryclass = {astro-ph.CO},
	title = {{The completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: measurement of the BAO and growth rate of structure of the luminous red galaxy sample from the anisotropic correlation function between redshifts 0.6 and 1}},
	volume = {500},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa2800}}

@article{GilMarin20,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.498.2492G},
	archiveprefix = {arXiv},
	author = {{Gil-Mar{\'\i}n}, H{\'e}ctor and others},
	doi = {10.1093/mnras/staa2455},
	eprint = {2007.08994},
	journal = {\mnras},
	keywords = {cosmological parameters, large-scale structure of the Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	number = {2},
	pages = {2492-2531},
	primaryclass = {astro-ph.CO},
	title = {{The Completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: measurement of the BAO and growth rate of structure of the luminous red galaxy sample from the anisotropic power spectrum between redshifts 0.6 and 1.0}},
	volume = {498},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa2455}}

@ARTICLE{enet,
       author = {{Weaverdyck}, Noah and {Huterer}, Dragan},
        title = "{Mitigating contamination in LSS surveys: a comparison of methods}",
      journal = {\mnras},
     keywords = {methods: data analysis, methods: statistical, surveys, cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = may,
       volume = {503},
       number = {4},
        pages = {5061-5084},
          doi = {10.1093/mnras/stab709},
archivePrefix = {arXiv},
       eprint = {2007.14499},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.503.5061W},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@ARTICLE{unWISE,
       author = {{Meisner}, A.~M. and {Lang}, D. and {Schlegel}, D.~J.},
        title = "{Deep Full-sky Coadds from Three Years of WISE and NEOWISE Observations}",
      journal = {\aj},
     keywords = {infrared: general, techniques: image processing, Astrophysics - Instrumentation and Methods for Astrophysics},
         year = 2017,
        month = oct,
       volume = {154},
       number = {4},
          eid = {161},
        pages = {161},
          doi = {10.3847/1538-3881/aa894e},
archivePrefix = {arXiv},
       eprint = {1705.06746},
 primaryClass = {astro-ph.IM},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2017AJ....154..161M},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{Krolewski:2019yrv,
  title={unWISE tomography of Planck CMB lensing},
  author={Krolewski, Alex and Ferraro, Simone and Schlafly, Edward F and White, Martin},
  journal={Journal of Cosmology and Astroparticle Physics},
  volume={2020},
  number={05},
  pages={047},
  year={2020},
  publisher={IOP Publishing}
}

@article{LSST:2019wqx,
  title={Tomographic galaxy clustering with the Subaru Hyper Suprime-Cam first year public data release},
  author={Nicola, Andrina and Alonso, David and S{\'a}nchez, Javier and Slosar, An{\v{z}}e and Awan, Humna and Broussard, Adam and Dunkley, Jo and Gawiser, Eric and Gomes, Zahra and Mandelbaum, Rachel and others},
  journal={Journal of Cosmology and Astroparticle Physics},
  volume={2020},
  number={03},
  pages={044},
  year={2020},
  publisher={IOP Publishing}
}

%Robert and {Freedman}, Wendy L. and {Hewitt}, Jacqueline and {Hu}, Wayne and {Huth}, John and {Kamionkowski}, Marc and {Kolb}, Edward W. and {Knox}, Lloyd and {Mather}, John C. and {Staggs}, Suzanne and {Suntzeff}, Nicholas B.},
@ARTICLE{DEtaskforce,
       author = {{Albrecht}, Andreas and {Bernstein}, Gary and {Cahn}, Robert
                  and {et al.}
        },
        title = "{Report of the Dark Energy Task Force}",
      journal = {arXiv e-prints},
     keywords = {Astrophysics},
         year = 2006,
        month = sep,
          eid = {astro-ph/0609591},
        pages = {astro-ph/0609591},
          doi = {10.48550/arXiv.astro-ph/0609591},
archivePrefix = {arXiv},
       eprint = {astro-ph/0609591},
 primaryClass = {astro-ph},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2006astro.ph..9591A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{DESoverview,
       author = {{Dark Energy Survey Collaboration} },
        title = "{The Dark Energy Survey: more than dark energy - an overview}",
      journal = {\mnras},
     keywords = {surveys, minor planets, asteroids: general, supernovae: general, Galaxy: general, galaxies: general, quasars: general, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies},
         year = 2016,
        month = aug,
       volume = {460},
       number = {2},
        pages = {1270-1299},
          doi = {10.1093/mnras/stw641},
archivePrefix = {arXiv},
       eprint = {1601.00329},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.460.1270D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}
%and {Abbott}, T. and {Abdalla}, F.~B. and {Aleksi{\'c}}, J. and {Allam}, S. and {Amara}, A. and {Bacon}, D. and {Balbinot}, E. and {Banerji}, M. and {Bechtol}, K. and {Benoit-L{\'e}vy}, A. and {Bernstein}, G.~M. and {Bertin}, E. and {Blazek}, J. and {Bonnett}, C. and {Bridle}, S. and {Brooks}, D. and {Brunner}, R.~J. and {Buckley-Geer}, E. and {Burke}, D.~L. and {Caminha}, G.~B. and {Capozzi}, D. and {Carlsen}, J. and {Carnero-Rosell}, A. and {Carollo}, M. and {Carrasco-Kind}, M. and {Carretero}, J. and {Castander}, F.~J. and {Clerkin}, L. and {Collett}, T. and {Conselice}, C. and {Crocce}, M. and {Cunha}, C.~E. and {D'Andrea}, C.~B. and {da Costa}, L.~N. and {Davis}, T.~M. and {Desai}, S. and {Diehl}, H.~T. and {Dietrich}, J.~P. and {Dodelson}, S. and {Doel}, P. and {Drlica-Wagner}, A. and {Estrada}, J. and {Etherington}, J. and {Evrard}, A.~E. and {Fabbri}, J. and {Finley}, D.~A. and {Flaugher}, B. and {Foley}, R.~J. and {Fosalba}, P. and {Frieman}, J. and {Garc{\'\i}a-Bellido}, J. and {Gaztanaga}, E. and {Gerdes}, D.~W. and {Giannantonio}, T. and {Goldstein}, D.~A. and {Gruen}, D. and {Gruendl}, R.~A. and {Guarnieri}, P. and {Gutierrez}, G. and {Hartley}, W. and {Honscheid}, K. and {Jain}, B. and {James}, D.~J. and {Jeltema}, T. and {Jouvel}, S. and {Kessler}, R. and {King}, A. and {Kirk}, D. and {Kron}, R. and {Kuehn}, K. and {Kuropatkin}, N. and {Lahav}, O. and {Li}, T.~S. and {Lima}, M. and {Lin}, H. and {Maia}, M.~A.~G. and {Makler}, M. and {Manera}, M. and {Maraston}, C. and {Marshall}, J.~L. and {Martini}, P. and {McMahon}, R.~G. and {Melchior}, P. and {Merson}, A. and {Miller}, C.~J. and {Miquel}, R. and {Mohr}, J.~J. and {Morice-Atkinson}, X. and {Naidoo}, K. and {Neilsen}, E. and {Nichol}, R.~C. and {Nord}, B. and {Ogando}, R. and {Ostrovski}, F. and {Palmese}, A. and {Papadopoulos}, A. and {Peiris}, H.~V. and {Peoples}, J. and {Percival}, W.~J. and {Plazas}, A.~A. and {Reed}, S.~L. and {Refregier}, A. and {Romer}, A.~K. and {Roodman}, A. and {Ross}, A. and {Rozo}, E. and {Rykoff}, E.~S. and {Sadeh}, I. and {Sako}, M. and {S{\'a}nchez}, C. and {Sanchez}, E. and {Santiago}, B. and {Scarpine}, V. and {Schubnell}, M. and {Sevilla-Noarbe}, I. and {Sheldon}, E. and {Smith}, M. and {Smith}, R.~C. and {Soares-Santos}, M. and {Sobreira}, F. and {Soumagnac}, M. and {Suchyta}, E. and {Sullivan}, M. and {Swanson}, M. and {Tarle}, G. and {Thaler}, J. and {Thomas}, D. and {Thomas}, R.~C. and {Tucker}, D. and {Vieira}, J.~D. and {Vikram}, V. and {Walker}, A.~R. and {Wechsler}, R.~H. and {Weller}, J. and {Wester}, W. and {Whiteway}, L. and {Wilcox}, H. and {Yanny}, B. and {Zhang}, Y. and {Zuntz}, J.





@ARTICLE{Flaugher,
       author = {{Flaugher}, B. and {Diehl}, H.~T. and {Honscheid}, K. and {DES} },
        title = "{The Dark Energy Camera}",
      journal = {\aj},
     keywords = {atlases, catalogs, cosmology: observations, instrumentation: detectors, instrumentation: photometers, surveys, Astrophysics - Instrumentation and Methods for Astrophysics},
         year = 2015,
        month = nov,
       volume = {150},
       number = {5},
          eid = {150},
        pages = {150},
          doi = {10.1088/0004-6256/150/5/150},
archivePrefix = {arXiv},
       eprint = {1504.02900},
 primaryClass = {astro-ph.IM},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2015AJ....150..150F},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}
%{Abbott}, T.~M.~C. and {Alvarez}, O. and {Angstadt}, R. and {Annis}, J.~T. and {Antonik}, M. and {Ballester}, O. and {Beaufore}, L. and {Bernstein}, G.~M. and {Bernstein}, R.~A. and {Bigelow}, B. and {Bonati}, M. and {Boprie}, D. and {Brooks}, D. and {Buckley-Geer}, E.~J. and {Campa}, J. and {Cardiel-Sas}, L. and {Castander}, F.~J. and {Castilla}, J. and {Cease}, H. and {Cela-Ruiz}, J.~M. and {Chappa}, S. and {Chi}, E. and {Cooper}, C. and {da Costa}, L.~N. and {Dede}, E. and {Derylo}, G. and {DePoy}, D.~L. and {de Vicente}, J. and {Doel}, P. and {Drlica-Wagner}, A. and {Eiting}, J. and {Elliott}, A.~E. and {Emes}, J. and {Estrada}, J. and {Fausti Neto}, A. and {Finley}, D.~A. and {Flores}, R. and {Frieman}, J. and {Gerdes}, D. and {Gladders}, M.~D. and {Gregory}, B. and {Gutierrez}, G.~R. and {Hao}, J. and {Holland}, S.~E. and {Holm}, S. and {Huffman}, D. and {Jackson}, C. and {James}, D.~J. and {Jonas}, M. and {Karcher}, A. and {Karliner}, I. and {Kent}, S. and {Kessler}, R. and {Kozlovsky}, M. and {Kron}, R.~G. and {Kubik}, D. and {Kuehn}, K. and {Kuhlmann}, S. and {Kuk}, K. and {Lahav}, O. and {Lathrop}, A. and {Lee}, J. and {Levi}, M.~E. and {Lewis}, P. and {Li}, T.~S. and {Mandrichenko}, I. and {Marshall}, J.~L. and {Martinez}, G. and {Merritt}, K.~W. and {Miquel}, R. and {Mu{\~n}oz}, F. and {Neilsen}, E.~H. and {Nichol}, R.~C. and {Nord}, B. and {Ogando}, R. and {Olsen}, J. and {Palaio}, N. and {Patton}, K. and {Peoples}, J. and {Plazas}, A.~A. and {Rauch}, J. and {Reil}, K. and {Rheault}, J. -P. and {Roe}, N.~A. and {Rogers}, H. and {Roodman}, A. and {Sanchez}, E. and {Scarpine}, V. and {Schindler}, R.~H. and {Schmidt}, R. and {Schmitt}, R. and {Schubnell}, M. and {Schultz}, K. and {Schurter}, P. and {Scott}, L. and {Serrano}, S. and {Shaw}, T.~M. and {Smith}, R.~C. and {Soares-Santos}, M. and {Stefanik}, A. and {Stuermer}, W. and {Suchyta}, E. and {Sypniewski}, A. and {Tarle}, G. and {Thaler}, J. and {Tighe}, R. and {Tran}, C. and {Tucker}, D. and {Walker}, A.~R. and {Wang}, G. and {Watson}, M. and {Weaverdyck}, C. and {Wester}, W. and {Woods}, R. and {Yanny}, B. and {DES Collaboration}



@ARTICLE{combination_eBOSS,
       author = {{Dumerchat}, Tyann and {Bautista}, Julian E.},
        title = "{Baryon acoustic oscillations from a joint analysis of the large-scale clustering in Fourier and configuration space}",
      journal = {\aap},
     keywords = {cosmological parameters, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = nov,
       volume = {667},
          eid = {A80},
        pages = {A80},
          doi = {10.1051/0004-6361/202244100},
archivePrefix = {arXiv},
       eprint = {2209.06011},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022A&A...667A..80D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{Peebles1970,
       author = {{Peebles}, P.~J.~E. and {Yu}, J.~T.},
        title = "{Primeval Adiabatic Perturbation in an Expanding Universe}",
      journal = {\apj},
         year = 1970,
        month = dec,
       volume = {162},
        pages = {815},
          doi = {10.1086/150713},
       adsurl = {https://ui.adsabs.harvard.edu/abs/1970ApJ...162..815P},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{SZ1970,
       author = {{Sunyaev}, R.~A. and {Zeldovich}, Ya. B.},
        title = "{Small-Scale Fluctuations of Relic Radiation}",
      journal = {APSS},
         year = 1970,
        month = apr,
       volume = {7},
       number = {1},
        pages = {3-19},
          doi = {10.1007/BF00653471},
       adsurl = {https://ui.adsabs.harvard.edu/abs/1970Ap&SS...7....3S},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




@ARTICLE{Bond1984,
       author = {{Bond}, J.~R. and {Efstathiou}, G.},
        title = "{Cosmic background radiation anisotropies in universes dominated by nonbaryonic dark matter}",
      journal = {\apjl},
     keywords = {Background Radiation, Big Bang Cosmology, Dark Matter, Elementary Particle Interactions, Galactic Evolution, Mass To Light Ratios, Relic Radiation, Anisotropy, Brightness Temperature, Galactic Clusters, Linear Polarization, Mass Distribution, Neutrinos, Space Radiation},
         year = 1984,
        month = oct,
       volume = {285},
        pages = {L45-L48},
          doi = {10.1086/184362},
       adsurl = {https://ui.adsabs.harvard.edu/abs/1984ApJ...285L..45B},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{Bond1987,
       author = {{Bond}, J.~R. and {Efstathiou}, G.},
        title = "{The statistics of cosmic background radiation fluctuations}",
      journal = {\mnras},
     keywords = {Computational Astrophysics, Cosmology, Radiation Distribution, Relic Radiation, Statistical Analysis, Angular Correlation, Anisotropy, Correlation, Dark Matter, Power Spectra, Space Radiation},
         year = 1987,
        month = jun,
       volume = {226},
        pages = {655-687},
          doi = {10.1093/mnras/226.3.655},
       adsurl = {https://ui.adsabs.harvard.edu/abs/1987MNRAS.226..655B},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}


@ARTICLE{CMB_BAO_1,
       author = {{Torbet}, E. and {Devlin}, M.~J. and {Dorwart}, W.~B. and {Herbig}, T. and {Miller}, A.~D. and {Nolta}, M.~R. and {Page}, L. and {Puchalla}, J. and {Tran}, H.~T.},
        title = "{A Measurement of the Angular Power Spectrum of the Microwave Background Made from the High Chilean Andes}",
      journal = {\apjl},
     keywords = {ATMOSPHERIC EFFECTS, COSMOLOGY: COSMIC MICROWAVE BACKGROUND, COSMOLOGY: OBSERVATIONS, Atmospheric Effects, Cosmology: Cosmic Microwave Background, Cosmology: Observations, Astrophysics},
         year = 1999,
        month = aug,
       volume = {521},
       number = {2},
        pages = {L79-L82},
          doi = {10.1086/312197},
archivePrefix = {arXiv},
       eprint = {astro-ph/9905100},
 primaryClass = {astro-ph},
       adsurl = {https://ui.adsabs.harvard.edu/abs/1999ApJ...521L..79T},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}


@ARTICLE{Boomerang_BAO_peak,
       author = {{Mauskopf}, P.~D. and others},
        title = "{Measurement of a Peak in the Cosmic Microwave Background Power Spectrum from the North American Test Flight of Boomerang}",
      journal = {\apjl},
     keywords = {Balloons, Cosmology: Cosmic Microwave Background, Cosmology: Observations, Instrumentation: Photometers, Methods: Data Analysis, Telescopes, Astrophysics, General Relativity and Quantum Cosmology, High Energy Physics - Experiment},
         year = 2000,
        month = jun,
       volume = {536},
       number = {2},
        pages = {L59-L62},
          doi = {10.1086/312743},
archivePrefix = {arXiv},
       eprint = {astro-ph/9911444},
 primaryClass = {astro-ph},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2000ApJ...536L..59M},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{Maxima_BAO_Peak,
       author = {{Hanany}, S. and {et al.}},
        title = "{MAXIMA-1: A Measurement of the Cosmic Microwave Background Anisotropy on Angular Scales of 10'-5{\textdegree}}",
      journal = {\apjl},
     keywords = {Cosmology: Cosmic Microwave Background, Cosmology: Observations, Astrophysics},
         year = 2000,
        month = dec,
       volume = {545},
       number = {1},
        pages = {L5-L9},
          doi = {10.1086/317322},
archivePrefix = {arXiv},
       eprint = {astro-ph/0005123},
 primaryClass = {astro-ph},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2000ApJ...545L...5H},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}







%{Abbott}, T.~M.~C. and {Alarcon}, A. and {Allam}, S. and {Andersen}, P. and {Andrade-Oliveira}, F. and {Annis}, J. and {Asorey}, J. and {Avila}, S. and {Bacon}, D. and {Banik}, N. and {Bassett}, B.~A. and {Baxter}, E. and {Bechtol}, K. and {Becker}, M.~R. and {Bernstein}, G.~M. and {Bertin}, E. and {Blazek}, J. and {Bridle}, S.~L. and {Brooks}, D. and {Brout}, D. and {Burke}, D.~L. and {Calcino}, J. and {Camacho}, H. and {Campos}, A. and {Carnero Rosell}, A. and {Carollo}, D. and {Carrasco Kind}, M. and {Carretero}, J. and {Castander}, F.~J. and {Cawthon}, R. and {Challis}, P. and {Chan}, K.~C. and {Chang}, C. and {Childress}, M. and {Crocce}, M. and {Cunha}, C.~E. and {D'Andrea}, C.~B. and {da Costa}, L.~N. and {Davis}, C. and {Davis}, T.~M. and {De Vicente}, J. and {DePoy}, D.~L. and {DeRose}, J. and {Desai}, S. and {Diehl}, H.~T. and {Dietrich}, J.~P. and {Dodelson}, S. and {Doel}, P. and {Drlica-Wagner}, A. and {Eifler}, T.~F. and {Elvin-Poole}, J. and {Estrada}, J. and {Evrard}, A.~E. and {Fernandez}, E. and {Flaugher}, B. and {Foley}, R.~J. and {Fosalba}, P. and {Frieman}, J. and {Galbany}, L. and {Garc{\'\i}a-Bellido}, J. and {Gatti}, M. and {Gaztanaga}, E. and {Gerdes}, D.~W. and {Giannantonio}, T. and {Glazebrook}, K. and {Goldstein}, D.~A. and {Gruen}, D. and {Gruendl}, R.~A. and {Gschwend}, J. and {Gutierrez}, G. and {Hartley}, W.~G. and {Hinton}, S.~R. and {Hollowood}, D.~L. and {Honscheid}, K. and {Hoormann}, J.~K. and {Hoyle}, B. and {Huterer}, D. and {Jain}, B. and {James}, D.~J. and {Jarvis}, M. and {Jeltema}, T. and {Kasai}, E. and {Kent}, S. and {Kessler}, R. and {Kim}, A.~G. and {Kokron}, N. and {Krause}, E. and {Kron}, R. and {Kuehn}, K. and {Kuropatkin}, N. and {Lahav}, O. and {Lasker}, J. and {Lemos}, P. and {Lewis}, G.~F. and {Li}, T.~S. and {Lidman}, C. and {Lima}, M. and {Lin}, H. and {Macaulay}, E. and {MacCrann}, N. and {Maia}, M.~A.~G. and {March}, M. and {Marriner}, J. and {Marshall}, J.~L. and {Martini}, P. and {McMahon}, R.~G. and {Melchior}, P. and {Menanteau}, F. and {Miquel}, R. and {Mohr}, J.~J. and {Morganson}, E. and {Muir}, J. and {M{\"o}ller}, A. and {Neilsen}, E. and {Nichol}, R.~C. and {Nord}, B. and {Ogando}, R.~L.~C. and {Palmese}, A. and {Pan}, Y. -C. and {Peiris}, H.~V. and {Percival}, W.~J. and {Plazas}, A.~A. and {Porredon}, A. and {Prat}, J. and {Romer}, A.~K. and {Roodman}, A. and {Rosenfeld}, R. and {Ross}, A.~J. and {Rykoff}, E.~S. and {Samuroff}, S. and {S{\'a}nchez}, C. and {Sanchez}, E. and {Scarpine}, V. and {Schindler}, R. and {Schubnell}, M. and {Scolnic}, D. and {Secco}, L.~F. and {Serrano}, S. and {Sevilla-Noarbe}, I. and {Sharp}, R. and {Sheldon}, E. and {Smith}, M. and {Soares-Santos}, M. and {Sobreira}, F. and {Sommer}, N.~E. and {Swann}, E. and {Swanson}, M.~E.~C. and {Tarle}, G. and {Thomas}, D. and {Thomas}, R.~C. and {Troxel}, M.~A. and {Tucker}, B.~E. and {Uddin}, S.~A. and {Vielzeuf}, P. and {Walker}, A.~R. and {Wang}, M. and {Weaverdyck}, N. and {Wechsler}, R.~H. and {Weller}, J. and {Yanny}, B. and {Zhang}, B. and {Zhang}, Y. and {Zuntz}, J. and {DES Collaboration}
@ARTICLE{DES_all_probes_Y1,
       author = {{Dark Energy Survey Collaboration}
                },
        title = "{Cosmological Constraints from Multiple Probes in the Dark Energy Survey}",
      journal = {\prl},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2019,
        month = may,
       volume = {122},
       number = {17},
          eid = {171301},
        pages = {171301},
          doi = {10.1103/PhysRevLett.122.171301},
archivePrefix = {arXiv},
       eprint = {1811.02375},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2019PhRvL.122q1301A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}


  %              {Burtin}, Etienne and {Dawson}, Kyle S. and {Hou}, Jiamin and {Kneib}, Jean-Paul and {de Mattia}, Arnaud and {Percival}, Will J. and {Rossi}, Graziano and {Tojeiro}, Rita and {Zhao}, Cheng and {Zhao}, Gong-Bo and {Alam}, Shadab and {Brownstein}, Joel and {Chapman}, Michael J. and {Choi}, Peter D. and {Chuang}, Chia-Hsun and {Escoffier}, St{\'e}phanie and {de la Macorra}, Axel and {du Mas des Bourboux}, H{\'e}lion and {Mohammad}, Faizan G. and {Moon}, Jeongin and {M{\"u}ller}, Eva-Maria and {Nadathur}, Seshadri and {Newman}, Jeffrey A. and {Schneider}, Donald and {Seo}, Hee-Jong and {Wang}, Yuting

@ARTICLE{Bautista21,
       author = {{Bautista}, Julian E. and {Paviot}, Romain and {Vargas Maga{\~n}a}, Mariana and {de la Torre}, Sylvain and {Fromenteau}, Sebastien and {Gil-Mar{\'\i}n}, Hector and {Ross}, Ashley J. and eBOSS Collaboration
                },
        title = "{The completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: measurement of the BAO and growth rate of structure of the luminous red galaxy sample from the anisotropic correlation function between redshifts 0.6 and 1}",
      journal = {\mnras},
     keywords = {cosmology: dark energy, cosmology: large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = jan,
       volume = {500},
       number = {1},
        pages = {736-762},
          doi = {10.1093/mnras/staa2800},
archivePrefix = {arXiv},
       eprint = {2007.08993},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.500..736B},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@article{Tamone,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.499..210N},
	archiveprefix = {arXiv},
	author = {{Neveux}, Richard and others},
	doi = {10.1093/mnras/staa2780},
	eprint = {2007.08999},
	journal = {\mnras},
	keywords = {galaxies: distances and redshifts, dark energy, distance scale, large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = nov,
	number = {1},
	pages = {210-229},
	primaryclass = {astro-ph.CO},
	title = {{The completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: BAO and RSD measurements from the anisotropic power spectrum of the quasar sample between redshift 0.8 and 2.2}},
	volume = {499},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa2780}}

@article{2020MNRAS.499.5527T,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.499.5527T},
	archiveprefix = {arXiv},
	author = {{Tamone}, Am{\'e}lie and others},
	doi = {10.1093/mnras/staa3050},
	eprint = {2007.09009},
	journal = {\mnras},
	keywords = {galaxies: distances and redshifts, dark energy, distance scale, large-scale structure of Universe, cosmology: observations, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = dec,
	number = {4},
	pages = {5527-5546},
	primaryclass = {astro-ph.CO},
	title = {{The completed SDSS-IV extended baryon oscillation spectroscopic survey: growth rate of structure measurement from anisotropic clustering analysis in configuration space between redshift 0.6 and 1.1 for the emission-line galaxy sample}},
	volume = {499},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa3050}}

@ARTICLE{Manera13,
       author = {{Manera}, Marc and {et al.}},
        title = "{The clustering of galaxies in the SDSS-III Baryon Oscillation Spectroscopic Survey: a large sample of mock galaxy catalogues}",
      journal = {\mnras},
     keywords = {galaxies: haloes, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2013,
        month = jan,
       volume = {428},
       number = {2},
        pages = {1036-1054},
          doi = {10.1093/mnras/sts084},
archivePrefix = {arXiv},
       eprint = {1203.6609},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2013MNRAS.428.1036M},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

%{Chuang}, Chia-Hsun and {Bautista}, Julian and {de Mattia}, Arnaud and {Raichoor}, Anand and {Ross}, Ashley J. and {Hou}, Jiamin and {Neveux}, Richard and {Tao}, Charling and {Burtin}, Etienne and {Dawson}, Kyle S. and {de la Torre}, Sylvain and {Gil-Mar{\'\i}n}, H{\'e}ctor and {Kneib}, Jean-Paul and {Percival}, Will J. and {Rossi}, Graziano and {Tamone}, Am{\'e}lie and {Tinker}, Jeremy L. and {Zhao}, Gong-Bo and {Alam}, Shadab and {Mueller}, Eva-Maria}
@ARTICLE{Zhao21,
       author = {{Zhao}, Cheng and {et al.}},
        title = "{The completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: 1000 multi-tracer mock catalogues with redshift evolution and systematics for galaxies and quasars of the final data release}",
      journal = {\mnras},
     keywords = {methods: numerical, catalogues, cosmology: large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = may,
       volume = {503},
       number = {1},
        pages = {1149-1173},
          doi = {10.1093/mnras/stab510},
archivePrefix = {arXiv},
       eprint = {2007.08997},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.503.1149Z},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}


@ARTICLE{Leistedt13,
       author = {{Leistedt}, Boris and {Peiris}, Hiranya V. and {Mortlock}, Daniel J. and {Benoit-L{\'e}vy}, Aur{\'e}lien and {Pontzen}, Andrew},
        title = "{Estimating the large-scale angular power spectrum in the presence of systematics: a case study of Sloan Digital Sky Survey quasars}",
      journal = {\mnras},
     keywords = {galaxies: photometry, quasars: general, galaxies: statistics, large-scale structure of Universe, Astrophysics - Cosmology and Extragalactic Astrophysics},
         year = 2013,
        month = nov,
       volume = {435},
       number = {3},
        pages = {1857-1873},
          doi = {10.1093/mnras/stt1359},
archivePrefix = {arXiv},
       eprint = {1306.0005},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2013MNRAS.435.1857L},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

%{Cuesta}, Antonio and {Seo}, Hee-Jong and {de Putter}, Roland and {Ross}, Ashley J. and {White}, Martin and {Padmanabhan}, Nikhil and {Saito}, Shun and {Schlegel}, David J. and {Schlafly}, Eddie and {Seljak}, Uros and {Hern{\'a}ndez-Monteagudo}, Carlos and {S{\'a}nchez}, Ariel G. and {Percival}, Will J. and {Blanton}, Michael and {Skibba}, Ramin and {Schneider}, Don and {Reid}, Beth and {Mena}, Olga and {Viel}, Matteo and {Eisenstein}, Daniel J. and {Prada}, Francisco and {Weaver}, Benjamin A. and {Bahcall}, Neta and {Bizyaev}, Dimitry and {Brewinton}, Howard and {Brinkman}, Jon and {Nicolaci da Costa}, Luiz and {Gott}, John R. and {Malanushenko}, Elena and {Malanushenko}, Viktor and {Nichol}, Bob and {Oravetz}, Daniel and {Pan}, Kaike and {Palanque-Delabrouille}, Nathalie and {Ross}, Nicholas P. and {Simmons}, Audrey and {de Simoni}, Fernando and {Snedden}, Stephanie and {Yeche}, Christophe}
@ARTICLE{Ho12,
       author = {{Ho}, Shirley and {et al.}},
        title = "{Clustering of Sloan Digital Sky Survey III Photometric Luminous Galaxies: The Measurement, Systematics, and Cosmological Implications}",
      journal = {\apj},
     keywords = {cosmological parameters, dark energy, dark matter, distance scale, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2012,
        month = dec,
       volume = {761},
       number = {1},
          eid = {14},
        pages = {14},
          doi = {10.1088/0004-637X/761/1/14},
archivePrefix = {arXiv},
       eprint = {1201.2137},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2012ApJ...761...14H},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}






%and {Ho}, Shirley and {Padmanabhan}, Nikhil and {Percival}, Will J. and {Tinker}, Jeremy and {Tojeiro}, Rita and {White}, Martin and {Eisenstein}, Daniel J. and {Maraston}, Claudia and {Ross}, Ashley J. and {S{\'a}nchez}, Ariel G. and {Schlegel}, David and {Sheldon}, Erin and {Strauss}, Michael A. and {Thomas}, Daniel and {Wake}, David and {Beutler}, Florian and {Bizyaev}, Dmitry and {Bolton}, Adam S. and {Brownstein}, Joel R. and {Chuang}, Chia-Hsun and {Dawson}, Kyle and {Harding}, Paul and {Kitaura}, Francisco-Shu and {Leauthaud}, Alexie and {Masters}, Karen and {McBride}, Cameron K. and {More}, Surhud and {Olmstead}, Matthew D. and {Oravetz}, Daniel and {Nuza}, Sebasti{\'a}n E. and {Pan}, Kaike and {Parejko}, John and {Pforr}, Janine and {Prada}, Francisco and {Rodr{\'\i}guez-Torres}, Sergio and {Salazar-Albornoz}, Salvador and {Samushia}, Lado and {Schneider}, Donald P. and {Sc{\'o}ccola}, Claudia G. and {Simmons}, Audrey and {Vargas-Magana}, Mariana
@ARTICLE{BOSS_target,
       author = {{Reid}, Beth and {et al.}},
        title = "{SDSS-III Baryon Oscillation Spectroscopic Survey Data Release 12: galaxy target selection and large-scale structure catalogues}",
      journal = {\mnras},
     keywords = {cosmology: observations, (cosmology:) large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies},
         year = 2016,
        month = jan,
       volume = {455},
       number = {2},
        pages = {1553-1573},
          doi = {10.1093/mnras/stv2382},
archivePrefix = {arXiv},
       eprint = {1509.06529},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.1553R},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@ARTICLE{eBOSS_survey,
       author = {{Dawson}, Kyle S. and {et al.}},
        title = "{The SDSS-IV Extended Baryon Oscillation Spectroscopic Survey: Overview and Early Data}",
      journal = {\aj},
     keywords = {cosmology: observations, surveys, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2016,
        month = feb,
       volume = {151},
       number = {2},
          eid = {44},
        pages = {44},
          doi = {10.3847/0004-6256/151/2/44},
archivePrefix = {arXiv},
       eprint = {1508.04473},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2016AJ....151...44D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




@article{2021MNRAS.501.5616D,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.501.5616D},
	archiveprefix = {arXiv},
	author = {{de Mattia}, Arnaud and others},
	doi = {10.1093/mnras/staa3891},
	eprint = {2007.09008},
	journal = {\mnras},
	keywords = {galaxies: distances and redshifts, cosmology: observations, dark energy, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = mar,
	number = {4},
	pages = {5616-5645},
	primaryclass = {astro-ph.CO},
	title = {{The completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: measurement of the BAO and growth rate of structure of the emission line galaxy sample from the anisotropic power spectrum between redshift 0.6 and 1.1}},
	volume = {501},
	year = 2021,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa3891}}

@article{deMattia,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.500.1201H},
	archiveprefix = {arXiv},
	author = {{Hou}, Jiamin and others},
	doi = {10.1093/mnras/staa3234},
	eprint = {2007.08998},
	journal = {\mnras},
	keywords = {methods: data analysis, quasars: general, cosmology: observations, cosmology: large-scale structure of Universe, cosmology: dark energy, cosmology: distance scale, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jan,
	number = {1},
	pages = {1201-1221},
	primaryclass = {astro-ph.CO},
	title = {{The completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: BAO and RSD measurements from anisotropic clustering analysis of the quasar sample in configuration space between redshift 0.8 and 2.2}},
	volume = {500},
	year = 2021,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/staa3234}}

@article{2020ApJ...901..153D,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020ApJ...901..153D},
	archiveprefix = {arXiv},
	author = {{du Mas des Bourboux}, H{\'e}lion and others},
	doi = {10.3847/1538-4357/abb085},
	eid = {153},
	eprint = {2007.08995},
	journal = {\apj},
	keywords = {Dark energy, Cosmology, Large-scale structure of the universe, Baryon acoustic oscillations, Ly{\ensuremath{\alpha}} forest, 351, 343, 902, 138, 980, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = oct,
	number = {2},
	pages = {153},
	primaryclass = {astro-ph.CO},
	title = {{The Completed SDSS-IV Extended Baryon Oscillation Spectroscopic Survey: Baryon Acoustic Oscillations with Ly{\ensuremath{\alpha}} Forests}},
	volume = {901},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.3847/1538-4357/abb085}}

@article{Ross:2017,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2017MNRAS.464.1168R},
	archiveprefix = {arXiv},
	author = {{Ross}, Ashley J. and others},
	doi = {10.1093/mnras/stw2372},
	eprint = {1607.03145},
	journal = {\mnras},
	keywords = {cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jan,
	number = {1},
	pages = {1168-1191},
	primaryclass = {astro-ph.CO},
	title = {{The clustering of galaxies in the completed SDSS-III Baryon Oscillation Spectroscopic Survey: observational systematics and baryon acoustic oscillations in the correlation function}},
	volume = {464},
	year = 2017,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw2372}}

@article{Laurent:2017,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2017JCAP...07..017L},
	archiveprefix = {arXiv},
	author = {{Laurent}, Pierre and others},
	doi = {10.1088/1475-7516/2017/07/017},
	eid = {017},
	eprint = {1705.04718},
	journal = {\jcap},
	keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jul,
	number = {7},
	pages = {017},
	primaryclass = {astro-ph.CO},
	title = {{Clustering of quasars in SDSS-IV eBOSS: study of potential systematics and bias determination}},
	volume = {2017},
	year = 2017,
	bdsk-url-1 = {https://doi.org/10.1088/1475-7516/2017/07/017}}

@article{Bautista:2018,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2018ApJ...863..110B},
	archiveprefix = {arXiv},
	author = {{Bautista}, Julian E. and others},
	doi = {10.3847/1538-4357/aacea5},
	eid = {110},
	eprint = {1712.08064},
	journal = {\apj},
	keywords = {distance scale, large-scale structure of universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = aug,
	number = {1},
	pages = {110},
	primaryclass = {astro-ph.CO},
	title = {{The SDSS-IV Extended Baryon Oscillation Spectroscopic Survey: Baryon Acoustic Oscillations at Redshift of 0.72 with the DR14 Luminous Red Galaxy Sample}},
	volume = {863},
	year = 2018,
	bdsk-url-1 = {https://doi.org/10.3847/1538-4357/aacea5}}

@article{Icaza-Lizaola:2020,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.492.4189I},
	archiveprefix = {arXiv},
	author = {{Icaza-Lizaola}, M. and others},
	doi = {10.1093/mnras/stz3602},
	eprint = {1909.07742},
	journal = {\mnras},
	keywords = {surveys, dark energy, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = mar,
	number = {3},
	pages = {4189-4215},
	primaryclass = {astro-ph.CO},
	title = {{The clustering of the SDSS-IV extended Baryon Oscillation Spectroscopic Survey DR14 LRG sample: structure growth rate measurement from the anisotropic LRG correlation function in the redshift range 0.6 < z < 1.0}},
	volume = {492},
	year = 2020,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stz3602}}

@article{Vakili:2020,
	author = {{Vakili}, Mohammadjavad and others},
        title = "{Clustering of red sequence galaxies in the fourth data release of the Kilo-Degree Survey}",
      journal = {\aap},
     keywords = {galaxies: distances and redshifts, large-scale structure of Universe, methods: data analysis, methods: statistical, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2023,
        month = jul,
       volume = {675},
          eid = {A202},
        pages = {A202},
          doi = {10.1051/0004-6361/202039293},
archivePrefix = {arXiv},
       eprint = {2008.13154},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2023A&A...675A.202V},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{Weaverdyck_2021,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.503.5061W},
	archiveprefix = {arXiv},
	author = {{Weaverdyck}, Noah and {Huterer}, Dragan},
	doi = {10.1093/mnras/stab709},
	eprint = {2007.14499},
	journal = {\mnras},
	keywords = {methods: data analysis, methods: statistical, surveys, cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = may,
	number = {4},
	pages = {5061-5084},
	primaryclass = {astro-ph.CO},
	title = {{Mitigating contamination in LSS surveys: a comparison of methods}},
	volume = {503},
	year = 2021,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stab709}}

@article{2017MNRAS.464.4807H,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2017MNRAS.464.4807H},
	archiveprefix = {arXiv},
	author = {{Hinton}, Samuel R. and others},
	doi = {10.1093/mnras/stw2725},
	eprint = {1611.08040},
	journal = {\mnras},
	keywords = {cosmological parameters, dark energy, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = feb,
	number = {4},
	pages = {4807-4822},
	primaryclass = {astro-ph.CO},
	title = {{Measuring the 2D baryon acoustic oscillation signal of galaxies in WiggleZ: cosmological constraints}},
	volume = {464},
	year = 2017,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stw2725}}

@article{2014MNRAS.441.3524K,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2014MNRAS.441.3524K},
	archiveprefix = {arXiv},
	author = {{Kazin}, Eyal A. and others},
	doi = {10.1093/mnras/stu778},
	eprint = {1401.0358},
	journal = {\mnras},
	keywords = {cosmological parameters, distance scale, large-scale structure of the universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = jul,
	number = {4},
	pages = {3524-3542},
	primaryclass = {astro-ph.CO},
	title = {{The WiggleZ Dark Energy Survey: improved distance measurements to z = 1 with reconstruction of the baryonic acoustic feature}},
	volume = {441},
	year = 2014,
	bdsk-url-1 = {https://doi.org/10.1093/mnras/stu778}}

@article{2011MNRAS.418.1707B,
	adsnote = {Provided by the SAO/NASA Astrophysics Data System},
	adsurl = {https://ui.adsabs.harvard.edu/abs/2011MNRAS.418.1707B},
	archiveprefix = {arXiv},
	author = {{Blake}, Chris and others},
	doi = {10.1111/j.1365-2966.2011.19592.x},
	eprint = {1108.2635},
	journal = {\mnras},
	keywords = {surveys, cosmological parameters, distance scale, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
	month = dec,
	number = {3},
	pages = {1707-1724},
	primaryclass = {astro-ph.CO},
	title = {{The WiggleZ Dark Energy Survey: mapping the distance-redshift relation with baryon acoustic oscillations}},
	volume = {418},
	year = 2011,
	bdsk-url-1 = {https://doi.org/10.1111/j.1365-2966.2011.19592.x}}


@ARTICLE{Gschwend2018,
       author = {{Gschwend}, J. and DES},
        title = "{DES science portal: Computing photometric redshifts}",
      journal = {Astronomy and Computing},
     keywords = {Astronomical databases: catalogs, surveys, Methods: data analysis, Galaxies: distances and redshifts, statistics, Astrophysics - Astrophysics of Galaxies, Astrophysics - Instrumentation and Methods for Astrophysics},
         year = 2018,
        month = oct,
       volume = {25},
        pages = {58-80},
          doi = {10.1016/j.ascom.2018.08.008},
archivePrefix = {arXiv},
       eprint = {1708.05643},
 primaryClass = {astro-ph.GA},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2018A&C....25...58G},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}


@article{y3-balrog,
       author = {{Everett}, S. and DES},
        title = "{Dark Energy Survey Year 3 Results: Measuring the Survey Transfer Function with Balrog}",
      journal = {\apjs},
     keywords = {343, 1464, 351, 1857, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Instrumentation and Methods for Astrophysics},
         year = 2022,
        month = jan,
       volume = {258},
       number = {1},
          eid = {15},
        pages = {15},
          doi = {10.3847/1538-4365/ac26c1},
archivePrefix = {arXiv},
       eprint = {2012.12825},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022ApJS..258...15E},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{y3-shapecatalog,
    author = "Gatti, M. and Sheldon, E. and DES",
    title = "{Dark Energy Survey Year 3 Results: Weak Lensing Shape Catalogue}",
    eprint = "2011.03408",
    archivePrefix = "arXiv",
    primaryClass = "astro-ph.CO",
    reportNumber = "FERMILAB-PUB-20-545-AE, DES-2015-0048",
    doi = "10.1093/mnras/stab918",
    journal = "Mon. Not. Roy. Astron. Soc.",
    volume = "504",
    number = "3",
    pages = "4312-4336",
    year = "2021"
}

@article{y3-cosmicshear1,
        author = {{Amon}, A. and DES},
       title = "{Dark Energy Survey Year 3 results: Cosmology from cosmic shear and robustness to data calibration}",
  journal = {\prd},
  keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
  year = 2022,
  month = jan,
  volume = {105},
  number = {2},
  eid = {023514},
  pages = {023514},
  doi = {10.1103/PhysRevD.105.023514},
  archivePrefix = {arXiv},
  eprint = {2105.13543},
  primaryClass = {astro-ph.CO},
  adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.105b3514A},
  adsnote = {Provided by the SAO/NASA Astrophysics Data System}
        }

@article{y3-cosmicshear2,
        author = {{Secco}, L.~F. and {Samuroff}, S. and DES},
         title = "{Dark Energy Survey Year 3 results: Cosmology from cosmic shear and robustness to modeling uncertainty}",
 journal = {\prd},
 keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
 year = 2022,
 month = jan,
 volume = {105},
 number = {2},
 eid = {023515},
 pages = {023515},
 doi = {10.1103/PhysRevD.105.023515},
 archivePrefix = {arXiv},
 eprint = {2105.13544},
 primaryClass = {astro-ph.CO},
 adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.105b3515S},
 adsnote = {Provided by the SAO/NASA Astrophysics Data System}
        }

@ARTICLE{y1-clusters-kp,
       author = {{Abbott}, T.~M.~C. and {Aguena}, M. and {Alarcon}, A. and {Allam}, S. and {Allen}, S. and {Annis}, J. and {Avila}, S. and {Bacon}, D. and {Bechtol}, K. and {Bermeo}, A. and {Bernstein}, G.~M. and {Bertin}, E. and {Bhargava}, S. and {Bocquet}, S. and {Brooks}, D. and {Brout}, D. and {Buckley-Geer}, E. and {Burke}, D.~L. and {Carnero Rosell}, A. and {Carrasco Kind}, M. and {Carretero}, J. and {Castander}, F.~J. and {Cawthon}, R. and {Chang}, C. and {Chen}, X. and {Choi}, A. and {Costanzi}, M. and {Crocce}, M. and {da Costa}, L.~N. and {Davis}, T.~M. and {De Vicente}, J. and {DeRose}, J. and {Desai}, S. and {Diehl}, H.~T. and {Dietrich}, J.~P. and {Dodelson}, S. and {Doel}, P. and {Drlica-Wagner}, A. and {Eckert}, K. and {Eifler}, T.~F. and {Elvin-Poole}, J. and {Estrada}, J. and {Everett}, S. and {Evrard}, A.~E. and {Farahi}, A. and {Ferrero}, I. and {Flaugher}, B. and {Fosalba}, P. and {Frieman}, J. and {Garc{\'\i}a-Bellido}, J. and {Gatti}, M. and {Gaztanaga}, E. and {Gerdes}, D.~W. and {Giannantonio}, T. and {Giles}, P. and {Grandis}, S. and {Gruen}, D. and {Gruendl}, R.~A. and {Gschwend}, J. and {Gutierrez}, G. and {Hartley}, W.~G. and {Hinton}, S.~R. and {Hollowood}, D.~L. and {Honscheid}, K. and {Hoyle}, B. and {Huterer}, D. and {James}, D.~J. and {Jarvis}, M. and {Jeltema}, T. and {Johnson}, M.~W.~G. and {Johnson}, M.~D. and {Kent}, S. and {Krause}, E. and {Kron}, R. and {Kuehn}, K. and {Kuropatkin}, N. and {Lahav}, O. and {Li}, T.~S. and {Lidman}, C. and {Lima}, M. and {Lin}, H. and {MacCrann}, N. and {Maia}, M.~A.~G. and {Mantz}, A. and {Marshall}, J.~L. and {Martini}, P. and {Mayers}, J. and {Melchior}, P. and {Mena-Fern{\'a}ndez}, J. and {Menanteau}, F. and {Miquel}, R. and {Mohr}, J.~J. and {Nichol}, R.~C. and {Nord}, B. and {Ogando}, R.~L.~C. and {Palmese}, A. and {Paz-Chinch{\'o}n}, F. and {Plazas}, A.~A. and {Prat}, J. and {Rau}, M.~M. and {Romer}, A.~K. and {Roodman}, A. and {Rooney}, P. and {Rozo}, E. and {Rykoff}, E.~S. and {Sako}, M. and {Samuroff}, S. and {S{\'a}nchez}, C. and {Sanchez}, E. and {Saro}, A. and {Scarpine}, V. and {Schubnell}, M. and {Scolnic}, D. and {Serrano}, S. and {Sevilla-Noarbe}, I. and {Sheldon}, E. and {Smith}, J. Allyn. and {Smith}, M. and {Suchyta}, E. and {Swanson}, M.~E.~C. and {Tarle}, G. and {Thomas}, D. and {To}, C. and {Troxel}, M.~A. and {Tucker}, D.~L. and {Varga}, T.~N. and {von der Linden}, A. and {Walker}, A.~R. and {Wechsler}, R.~H. and {Weller}, J. and {Wilkinson}, R.~D. and {Wu}, H. and {Yanny}, B. and {Zhang}, Y. and {Zhang}, Z. and {Zuntz}, J. and {DES Collaboration}},
        title = "{Dark Energy Survey Year 1 Results: Cosmological constraints from cluster abundances and weak lensing}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2020,
        month = jul,
       volume = {102},
       number = {2},
          eid = {023509},
        pages = {023509},
          doi = {10.1103/PhysRevD.102.023509},
archivePrefix = {arXiv},
       eprint = {2002.11124},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2020PhRvD.102b3509A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




@article{y3-2x2ptaltlensresults,
        author = {{Porredon}, A. and DES},
        title = "{Dark Energy Survey Year 3 results: Cosmological constraints from galaxy clustering and galaxy-galaxy lensing using the MagLim lens sample}",
      journal = {\prd},
         year = 2022,
        month = nov,
       volume = {106},
       number = {10},
          eid = {103530},
        pages = {103530},
          doi = {10.1103/PhysRevD.106.103530},
archivePrefix = {arXiv},
       eprint = {2105.13546},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.106j3530P},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}	
        }

@ARTICLE{y3-2x2ptredmagic,
       author = {{Pandey}, S. and DES},
        title = "{Dark Energy Survey year 3 results: Constraints on cosmological parameters and galaxy-bias models from galaxy clustering and galaxy-galaxy lensing using the redMaGiC sample}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies},
         year = 2022,
        month = aug,
       volume = {106},
       number = {4},
          eid = {043520},
        pages = {043520},
          doi = {10.1103/PhysRevD.106.043520},
archivePrefix = {arXiv},
       eprint = {2105.13545},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022PhRvD.106d3520P},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



@ARTICLE{y3-sompz,
       author = {{Myles}, J. and DES},
        title = "{Dark Energy Survey Year 3 results: redshift calibration of the weak lensing source galaxies}",
      journal = {\mnras},
     keywords = {gravitational lensing: weak, galaxies: distances and redshifts, dark energy, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = aug,
       volume = {505},
       number = {3},
        pages = {4249-4277},
          doi = {10.1093/mnras/stab1515},
archivePrefix = {arXiv},
       eprint = {2012.08566},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.505.4249M},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{y3-6x2pt,
       author = {{DES} and {SPT Collaborations}},
        title = "{Joint analysis of Dark Energy Survey Year 3 data and CMB lensing from SPT and Planck. III. Combined cosmological constraints}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2023,
        month = jan,
       volume = {107},
       number = {2},
          eid = {023531},
        pages = {023531},
          doi = {10.1103/PhysRevD.107.023531},
archivePrefix = {arXiv},
       eprint = {2206.10824},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2023PhRvD.107b3531A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}



%Below:
%, C. and {Gruen}, D. and {DeRose}, J. and
         {Alarcon}, A. and {Bernstein}, G.~M. and {S{\'a}nchez}, C. and
         {Myles}, J. and {Roodman}, A. and {Allen}, S. and {Amon}, A. and
         {Choi}, A. and {Masters}, D.~C. and {Miquel}, R. and {Troxel}, M.~A. and
         {Wechsler}, R.~H. and {Abbott}, T.~M.~C. and {Annis}, J. and
         {Avila}, S. and {Bechtol}, K. and {Bridle}, S.~L. and {Brooks}, D. and
         {Buckley-Geer}, E. and {Burke}, D.~L. and {Carnero Rosell}, A. and
         {Carrasco Kind}, M. and {Carretero}, J. and {Castander}, F.~J. and
         {Cawthon}, R. and {D'Andrea}, C.~B. and {da Costa}, L.~N. and
         {De Vicente}, J. and {Desai}, S. and {Diehl}, H.~T. and {Doel}, P. and
         {Drlica-Wagner}, A. and {Eifler}, T.~F. and {Evrard}, A.~E. and
         {Flaugher}, B. and {Fosalba}, P. and {Frieman}, J. and
         {Garc{\'\i}a-Bellido}, J. and {Gaztanaga}, E. and {Gruendl}, R.~A. and
         {Gschwend}, J. and {Gutierrez}, G. and {Hartley}, W.~G. and
         {Hollowood}, D.~L. and {Honscheid}, K. and {James}, D.~J. and
         {Kuehn}, K. and {Kuropatkin}, N. and {Lima}, M. and {Lin}, H. and
         {Maia}, M.~A.~G. and {March}, M. and {Marshall}, J.~L. and
         {Melchior}, P. and {Menanteau}, F. and {Ogando}, R.~L.~C. and
         {Plazas}, A.~A. and {Rykoff}, E.~S. and {Sanchez}, E. and
         {Scarpine}, V. and {Serrano}, S. and {Sevilla-Noarbe}, I. and
         {Smith}, M. and {Soares-Santos}, M. and {Sobreira}, F. and
         {Suchyta}, E. and {Swanson}, M.~E.~C. and {Tarle}, G. and {Thomas}, D. and
         {Vikram}, V.

@article{y3-sompzbuzzard,
       author = {{Buchs}, R. and {Davis} and {DES}},
        title = "{Phenotypic redshifts with self-organizing maps: A novel method to characterize redshift distributions of source galaxies for weak lensing}",
      journal = {\mnras},
     keywords = {gravitational lensing: weak, galaxies: distances and redshifts, dark energy, Astrophysics - Cosmology and Nongalactic Astrophysics, Astrophysics - Astrophysics of Galaxies},
         year = 2019,
        month = oct,
       volume = {489},
       number = {1},
        pages = {820-841},
          doi = {10.1093/mnras/stz2162},
archivePrefix = {arXiv},
       eprint = {1901.05005},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2019MNRAS.489..820B},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{desy1-clustercosmology,
       author = {{Costanzi}, M. and {DES} and {SPT}},
        title = "{Cosmological constraints from DES Y1 cluster abundances and SPT multiwavelength data}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = feb,
       volume = {103},
       number = {4},
          eid = {043522},
        pages = {043522},
          doi = {10.1103/PhysRevD.103.043522},
archivePrefix = {arXiv},
       eprint = {2010.13800},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021PhRvD.103d3522C},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{y1-cluster6x2pt,
       author = {{To}, C. and {DES}},
        title = "{Dark Energy Survey Year 1 Results: Cosmological Constraints from Cluster Abundances, Weak Lensing, and Galaxy Correlations}",
      journal = {\prl},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = apr,
       volume = {126},
       number = {14},
          eid = {141301},
        pages = {141301},
          doi = {10.1103/PhysRevLett.126.141301},
archivePrefix = {arXiv},
       eprint = {2010.01138},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021PhRvL.126n1301T},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{y1-6x2pt,
       author = {{DES} and {SPT}},
        title = "{Dark Energy Survey year 1 results: Joint analysis of galaxy clustering, galaxy lensing, and CMB lensing two-point functions}",
      journal = {\prd},
     keywords = {Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2019,
        month = jul,
       volume = {100},
       number = {2},
          eid = {023541},
        pages = {023541},
          doi = {10.1103/PhysRevD.100.023541},
archivePrefix = {arXiv},
       eprint = {1810.02322},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2019PhRvD.100b3541A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}




% APS Extra stuff

@ARTICLE{2021JCAP...10..030G,
       author = {{Garc{\'\i}a-Garc{\'\i}a}, Carlos and {Ruiz-Zapatero}, Jaime and {Alonso}, David and {Bellini}, Emilio and {Ferreira}, Pedro G. and {Mueller}, Eva-Maria and {Nicola}, Andrina and {Ruiz-Lapuente}, Pilar},
        title = "{The growth of density perturbations in the last  10 billion years from tomographic large-scale structure data}",
      journal = {\jcap},
     keywords = {cosmological parameters from LSS, galaxy clustering, redshift surveys, weak gravitational lensing, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = oct,
       volume = {2021},
       number = {10},
          eid = {030},
        pages = {030},
          doi = {10.1088/1475-7516/2021/10/030},
archivePrefix = {arXiv},
       eprint = {2105.12108},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021JCAP...10..030G},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@book{wallisch2019cosmological,
  title={Cosmological probes of light relics},
  author={Wallisch, Benjamin},
  year={2019},
  publisher={Springer Nature}
}

@article{avery1996combining,
  title={Combining measurements with correlated errors},
  author={Avery, Paul},
  journal={CLEO Note CBX},
  pages={95--55},
  year={1996}
}

% Gil-Marin 2020 eBOSS LRGs

@ARTICLE{2020MNRAS.498.2492G,
       author = {{Gil-Mar{\'\i}n}, H{\'e}ctor and others},
        title = "{The Completed SDSS-IV extended Baryon Oscillation Spectroscopic Survey: measurement of the BAO and growth rate of structure of the luminous red galaxy sample from the anisotropic power spectrum between redshifts 0.6 and 1.0}",
      journal = {\mnras},
     keywords = {cosmological parameters, large-scale structure of the Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2020,
        month = oct,
       volume = {498},
       number = {2},
        pages = {2492-2531},
          doi = {10.1093/mnras/staa2455},
archivePrefix = {arXiv},
       eprint = {2007.08994},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2020MNRAS.498.2492G},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{ross2015information,
  title={The information content of anisotropic Baryon Acoustic Oscillation scale measurements},
  author={Ross, Ashley J and Percival, Will J and Manera, Marc},
  journal={Monthly Notices of the Royal Astronomical Society},
  volume={451},
  number={2},
  pages={1331--1340},
  year={2015},
  publisher={Oxford University Press}
}

% DES Harmonic Space Analyses

@ARTICLE{2021MNRAS.505.5714A,
       author = {{Andrade-Oliveira}, F. and {Camacho}, H. and {Faga}, L. and {DES}},
        title = "{Galaxy clustering in harmonic space from the dark energy survey year 1 data: compatibility with real-space results}",
      journal = {\mnras},
     keywords = {cosmology: observations, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2021,
        month = aug,
       volume = {505},
       number = {4},
        pages = {5714-5724},
          doi = {10.1093/mnras/stab1642},
archivePrefix = {arXiv},
       eprint = {2103.14190},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2021MNRAS.505.5714A},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{2022MNRAS.516.5799C,
       author = {{Camacho}, H. and {Andrade-Oliveira}, F. and {Troja}, A. and {DES}},
        title = "{Cosmic shear in harmonic space from the Dark Energy Survey Year 1 Data: compatibility with configuration space results}",
      journal = {\mnras},
     keywords = {cosmology: observations, (cosmology:) large-scale structure of Universe, gravitational lensing: weak, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = nov,
       volume = {516},
       number = {4},
        pages = {5799-5815},
          doi = {10.1093/mnras/stac2543},
archivePrefix = {arXiv},
       eprint = {2111.07203},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022MNRAS.516.5799C},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@ARTICLE{2022MNRAS.515.1942D,
       author = {{Doux}, C. and {Jain}, B. and {DES Collaboration}},
        title = "{Dark energy survey year 3 results: cosmological constraints from the analysis of cosmic shear in harmonic space}",
      journal = {\mnras},
     keywords = {gravitational lensing: weak, cosmological parameters, large-scale structure of Universe, Astrophysics - Cosmology and Nongalactic Astrophysics},
         year = 2022,
        month = sep,
       volume = {515},
       number = {2},
        pages = {1942-1972},
          doi = {10.1093/mnras/stac1826},
archivePrefix = {arXiv},
       eprint = {2203.07128},
 primaryClass = {astro-ph.CO},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2022MNRAS.515.1942D},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{Faga23,
	author = {{Faga}, L. and DES},
	journal = {To be submitted to MNRAS},
	title = {{DES-Y3 joint analysis of galaxy clustering and galaxy-galaxy lensing in harmonic space}},
	year = {2024}
}